models from EDA https://github.com/jasonwei20/eda_nlp/tree/d75e8bd4631f4d93260cb291aa47852d8eacd51d/experiments
some parameters changed - size of model(lstm, CNN) layers 

In [0]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
import keras.layers as layers
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping
import keras.backend as K
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, f1_score

import math
import time
import numpy as np
import random
from random import randint
random.seed(3)
import datetime, re, operator
from random import shuffle
from time import gmtime, strftime
import gc
import keras
import tensorflow as tf

import os
from os import listdir
from os.path import isfile, join, isdir
import pickle

Using TensorFlow backend.


In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13115621475811910657
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10465228947228632213
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7770808490716122124
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15701463552
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16989024056392473704
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/UniversalEmb

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/UniversalEmb


In [0]:
!rm crawl-300d-2M.vec.zip

In [0]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip

--2020-05-10 10:54:33--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4b8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  11.1MB/s    in 2m 12s  

2020-05-10 10:56:46 (11.0 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]



In [0]:
#!unzip crawl-300d-2M.vec.zip

Archive:  crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       


In [0]:
from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format('crawl-300d-2M.vec', binary=False) 

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Preproc data

In [0]:
def transform(data, w2i, n_classes, input_size):
  x_matrix = np.zeros((data.shape[0], input_size))
  y_matrix = np.zeros((data.shape[0], n_classes))
  sent_len = []
  found_word, unk_w = 0,0
  for i, line in enumerate(data.iterrows()):
    label = line[1]['lbl_index']
    sentence = line[1]['text']
    splited_sent = sentence.split(' ')
    sent_len.append(len(splited_sent))

    for enum, word in enumerate(splited_sent[:input_size]):
      if word in w2i:
        x_matrix[i][enum] = w2i[word]
        found_word+=1
      else:
        x_matrix[i][enum] = w2i['unk']
        unk_w+=1
    y_matrix[i][label] = 1.0
  print('Average sentence len: ', np.mean(sent_len))
  print('Max sentence len: ', np.max(sent_len) )
  print('{0:0.3f}% of tokens were replaced by unk token index'.format((unk_w/(unk_w+found_word))*100))

  return x_matrix.astype(int), y_matrix 


def create_train_index(tr, w2v_model, max_s_l=128, n_classes=None):
  w2i = {'pad': 0, 'unk': 1}

  #create tokens list and types
  tokens = []
  for indx, line in tr.iterrows():
    tokens.extend(line['text'].split(' '))
  types = list(set(tokens))
  #create w2i vocab and pretrained emb matrix
  emb_matrix = []
  emb_matrix.append(np.zeros(300)) #'pad'
  emb_matrix.append(np.zeros(300)) #'unk'
  s,d = 0,0
  for i, t in enumerate(types):
    if t in w2v_model:
      indx = len(w2i)
      w2i[t] = indx
      emb_matrix.append(w2v_model[t])
      d += 1
    else:
      s += 1
  print('Not found vectors {} out of {} unique words'.format(s, d+s))
  i2w = {j:i for i, j in w2i.items()}
  #transform train tokens to index in each text
  if n_classes is None:
    n_classes = len(tr.lbl_index.unique())
  x_tr, y_tr = transform(tr, w2i, n_classes, max_s_l)
  return x_tr, y_tr, w2i, i2w, np.array(emb_matrix)

In [0]:
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [0]:
def one_hot_to_categorical(y):
    assert len(y.shape) == 2
    return np.argmax(y, axis=1)

Models

In [0]:
import tensorflow as tf
#building the model in keras with embeddings
def build_lstm(word2vec_matrix, max_seq_len, num_classes):
  model = None
  model = Sequential()
  model.add(layers.Embedding(word2vec_matrix.shape[0], word2vec_matrix.shape[1], input_length=max_seq_len,  weights=[word2vec_matrix], trainable=False))
  model.add(Bidirectional(LSTM(256, return_sequences=True)))
  model.add(Dropout(0.5))
  model.add(Bidirectional(LSTM(128, return_sequences=False)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
  opt = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-07)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', f1 ])
  #print(model.summary())
  return model

In [0]:
#building the cnn in keras
def build_cnn(word2vec_matrix, max_seq_len, num_classes):
  model = None
  model = Sequential()
  model.add(layers.Embedding(word2vec_matrix.shape[0], word2vec_matrix.shape[1], input_length=max_seq_len,  weights=[word2vec_matrix]))
  model.add(layers.Conv1D(256, 5, activation='relu'))
  model.add(layers.GlobalMaxPooling1D())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
  opt = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-07)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', f1 ])
  return model

Run models

In [0]:
from keras.callbacks import History
def run_cnn(train_data, test_data, w2v_model, msl, n_c):

  #load data
  train_x, train_y, word2in, in2word, pretrained_vectors = create_train_index(train_data, w2v_model, msl)
  test_x, test_y = transform(test_data, word2in, len(train_data.lbl_index.unique()), train_x.shape[1])

  #initialize model
  model = build_cnn(pretrained_vectors, msl, n_c)

  #implement early stopping
  history = History()
  callbacks = [EarlyStopping(monitor='val_loss', patience=5), history]

   #implement early stopping
  history = History()
  callbacks = [EarlyStopping(monitor='val_loss', patience=5), history]

  #train model
  hist = model.fit(train_x, 
        train_y, 
        epochs=100000, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=512, 
        shuffle=True, 
        verbose=1)
  all_f1 = hist.history['val_f1']
  all_acc = hist.history['val_accuracy']

  print('Best F1:{0:0.3f}, best accuracy:{1:0.3f}'.format(max(all_f1), max(all_acc)))
  return max(all_f1), max(all_acc)

In [0]:
from keras.callbacks import History
def run_lstm(train_data, test_data, w2v_model, msl, n_c):

  #load data
  train_x, train_y, word2in, in2word, pretrained_vectors = create_train_index(train_data, w2v_model, msl)
  test_x, test_y = transform(test_data, word2in, len(train_data.lbl_index.unique()), train_x.shape[1])

  #initialize model
  model = build_lstm(pretrained_vectors, msl, n_c)

  #implement early stopping
  history = History()
  callbacks = [EarlyStopping(monitor='val_loss', patience=5), history]

  #train model
  hist = model.fit(train_x, 
        train_y, 
        epochs=100000, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=512, 
        shuffle=True, 
        verbose=1)
  all_f1 = hist.history['val_f1']
  all_acc = hist.history['val_accuracy']

  print('Best F1:{0:0.3f}, best accuracy:{1:0.3f}'.format(max(all_f1), max(all_acc)))
  return max(all_f1), max(all_acc)

#Trec-50

### 3000 

In [0]:
import pandas as pd
def read_trec50(file_name, typ=1):
  train = pd.read_csv(file_name, index_col=0)
  lbl2indx = {i:j for j, i in enumerate(train.labels.unique())}
  train.columns = ['text', 'labels']
  train['lbl_index'] = train.labels.apply(lambda x:lbl2indx[x])


  labels_t = [i.split(' ', 1)[0].split(':')[typ] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  test  = pd.DataFrame({'text':texts_t, 'labels': labels_t})
  test['lbl_index'] = test.labels.apply(lambda x:lbl2indx[x])

  return train, test, lbl2indx
  

In [0]:
number_of_attemp_to_fit = 3

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_3k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 132 out of 6350 unique words
Average sentence len:  10.240343347639485
Max sentence len:  32
1.802% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
12.320% of tokens were replaced by unk token index
Train on 3029 samples, validate on 500 samples
Epoch 1/100000
3029/3029 [==============================] - 3s 901us/step - loss: 3.7821 - accuracy: 0.1014 - f1: 0.0000e+00 - val_loss: 3.5443 - val_accuracy: 0.1100 - val_f1: 0.0000e+00
Epoch 2/100000
3029/3029 [==============================] - 2s 530us/step - loss: 3.4679 - accuracy: 0.1367 - f1: 0.0000e+00 - val_loss: 3.2471 - val_accuracy: 0.2240 - val_f1: 0.0000e+00
Epoch 3/100000
3029/3029 [==============================] - 2s 523us/step - loss: 3.3442 - accuracy: 0.1588 - f1: 0.0000e+00 - val_loss: 3.1232 - val_accuracy: 0.2460 - val_f1: 0.0000e+00
Epoch 4/100000
3029/3029 [==============================] - 2s 521us/step - loss: 3.2197 - accuracy: 0.2020 - f1: 0.0197 - val

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 132 out of 6350 unique words
Average sentence len:  10.240343347639485
Max sentence len:  32
1.802% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
12.320% of tokens were replaced by unk token index
Train on 3029 samples, validate on 500 samples
Epoch 1/100000
3029/3029 [==============================] - 0s 93us/step - loss: 3.6291 - accuracy: 0.1225 - f1: 0.0000e+00 - val_loss: 3.2968 - val_accuracy: 0.2360 - val_f1: 0.0000e+00
Epoch 2/100000
3029/3029 [==============================] - 0s 41us/step - loss: 3.1865 - accuracy: 0.1885 - f1: 7.0922e-04 - val_loss: 2.9458 - val_accuracy: 0.4400 - val_f1: 0.0000e+00
Epoch 3/100000
3029/3029 [==============================] - 0s 37us/step - loss: 2.8845 - accuracy: 0.3186 - f1: 0.0027 - val_loss: 2.6896 - val_accuracy: 0.5020 - val_f1: 0.0040
Epoch 4/100000
3029/3029 [==============================] - 0s 33us/step - loss: 2.5109 - accuracy: 0.3849 - f1: 0.1182 - val_loss: 2.312

### 4000k

In [0]:
number_of_attemp_to_fit = 3

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_4k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 171 out of 7707 unique words
Average sentence len:  10.186639979984989
Max sentence len:  37
1.987% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
11.336% of tokens were replaced by unk token index
Train on 3997 samples, validate on 500 samples
Epoch 1/100000
3997/3997 [==============================] - 3s 817us/step - loss: 3.6224 - accuracy: 0.1529 - f1: 0.0000e+00 - val_loss: 3.3555 - val_accuracy: 0.1100 - val_f1: 0.0000e+00
Epoch 2/100000
3997/3997 [==============================] - 2s 585us/step - loss: 3.1959 - accuracy: 0.1636 - f1: 0.0000e+00 - val_loss: 3.0915 - val_accuracy: 0.1100 - val_f1: 0.0000e+00
Epoch 3/100000
3997/3997 [==============================] - 2s 562us/step - loss: 3.0118 - accuracy: 0.1979 - f1: 6.0386e-04 - val_loss: 2.9892 - val_accuracy: 0.2160 - val_f1: 0.0000e+00
Epoch 4/100000
3997/3997 [==============================] - 2s 560us/step - loss: 2.8231 - accuracy: 0.2735 - f1: 0.0735 - val

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 171 out of 7707 unique words
Average sentence len:  10.186639979984989
Max sentence len:  37
1.987% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
11.336% of tokens were replaced by unk token index
Train on 3997 samples, validate on 500 samples
Epoch 1/100000
3997/3997 [==============================] - 0s 87us/step - loss: 3.3881 - accuracy: 0.1479 - f1: 0.0000e+00 - val_loss: 3.0289 - val_accuracy: 0.3920 - val_f1: 0.0000e+00
Epoch 2/100000
3997/3997 [==============================] - 0s 32us/step - loss: 2.8624 - accuracy: 0.3247 - f1: 0.0190 - val_loss: 2.6770 - val_accuracy: 0.4540 - val_f1: 0.0119
Epoch 3/100000
3997/3997 [==============================] - 0s 34us/step - loss: 2.4359 - accuracy: 0.4058 - f1: 0.1709 - val_loss: 2.2796 - val_accuracy: 0.4980 - val_f1: 0.4299
Epoch 4/100000
3997/3997 [==============================] - 0s 35us/step - loss: 2.0264 - accuracy: 0.4899 - f1: 0.3862 - val_loss: 1.9399 - val_

### 5000

In [0]:
number_of_attemp_to_fit = 3

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_5k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 200 out of 8924 unique words
Average sentence len:  10.208824116590138
Max sentence len:  37
1.918% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
10.378% of tokens were replaced by unk token index
Train on 5009 samples, validate on 500 samples
Epoch 1/100000
5009/5009 [==============================] - 4s 774us/step - loss: 3.5453 - accuracy: 0.1523 - f1: 0.0000e+00 - val_loss: 3.2598 - val_accuracy: 0.1140 - val_f1: 0.0000e+00
Epoch 2/100000
5009/5009 [==============================] - 3s 543us/step - loss: 3.0566 - accuracy: 0.1976 - f1: 0.0000e+00 - val_loss: 3.0245 - val_accuracy: 0.2380 - val_f1: 0.0000e+00
Epoch 3/100000
5009/5009 [==============================] - 3s 548us/step - loss: 2.7919 - accuracy: 0.2801 - f1: 0.0323 - val_loss: 2.8363 - val_accuracy: 0.2460 - val_f1: 0.1096
Epoch 4/100000
5009/5009 [==============================] - 3s 526us/step - loss: 2.5684 - accuracy: 0.3354 - f1: 0.1684 - val_loss: 2

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 200 out of 8924 unique words
Average sentence len:  10.208824116590138
Max sentence len:  37
1.918% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
10.378% of tokens were replaced by unk token index
Train on 5009 samples, validate on 500 samples
Epoch 1/100000
5009/5009 [==============================] - 0s 82us/step - loss: 3.3830 - accuracy: 0.1435 - f1: 0.0000e+00 - val_loss: 2.9972 - val_accuracy: 0.3980 - val_f1: 0.0000e+00
Epoch 2/100000
5009/5009 [==============================] - 0s 36us/step - loss: 2.6794 - accuracy: 0.3627 - f1: 0.0453 - val_loss: 2.5395 - val_accuracy: 0.4780 - val_f1: 0.1238
Epoch 3/100000
5009/5009 [==============================] - 0s 34us/step - loss: 2.1614 - accuracy: 0.4712 - f1: 0.3153 - val_loss: 2.1591 - val_accuracy: 0.4940 - val_f1: 0.5087
Epoch 4/100000
5009/5009 [==============================] - 0s 33us/step - loss: 1.7288 - accuracy: 0.5708 - f1: 0.4975 - val_loss: 1.7836 - val_

### TREC-50 with our augmetation

### 3000

### 4000

### 5000

In [0]:
def read_aug(name, lbl2indx):
  aug = pd.read_csv(name, index_col=0)
  aug['lbl_index'] = aug.labels.apply(lambda x:lbl2indx[x])
  aug = aug[['texts', 'lbl_index']]
  aug.columns = ['text', 'lbl_index']
  return aug

In [0]:
number_of_attemp_to_fit = 1

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_5k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 128, len(train_data.lbl_index.unique())

In [0]:
augmentation = read_aug('trec50_n1_bert_squad_pr_b-0.5_min_b-min_max_b-max_coefs2-2.csv', l2i)

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([train_data,augmentation]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 27605 out of 65709 unique words
Average sentence len:  10.322054778517645
Max sentence len:  25601
12.171% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
4.683% of tokens were replaced by unk token index
Train on 94161 samples, validate on 500 samples
Epoch 1/100000
94161/94161 [==============================] - 159s 2ms/step - loss: 2.1130 - accuracy: 0.4713 - f1: 0.3077 - val_loss: 2.6949 - val_accuracy: 0.3460 - val_f1: 0.3784
Epoch 2/100000
94161/94161 [==============================] - 162s 2ms/step - loss: 1.6283 - accuracy: 0.5768 - f1: 0.5204 - val_loss: 1.9905 - val_accuracy: 0.6020 - val_f1: 0.3939
Epoch 3/100000
94161/94161 [==============================] - 160s 2ms/step - loss: 1.3355 - accuracy: 0.6429 - f1: 0.6097 - val_loss: 1.6804 - val_accuracy: 0.6820 - val_f1: 0.6239
Epoch 4/100000
94161/94161 [==============================] - 159s 2ms/step - loss: 1.0911 - accuracy: 0.7098 - f1: 0.7000 - val_loss: 1.

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([train_data,augmentation]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 27605 out of 65709 unique words
Average sentence len:  10.322054778517645
Max sentence len:  25601
12.171% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
4.683% of tokens were replaced by unk token index
Best F1:0.857, best accuracy:0.854
f1_average: 0.8565487861633301, std:0.0
acc_average: 0.8539999723434448, std:0.0


## Trec-50 with EDA

### 5000

In [0]:
def read_eda_trec50(file_name, typ = 1):
  "type = 0 - trec6, type = 1 - trec50"
  labels = [i[:-1].split('\t' )[0] for i in open(file_name, encoding = 'windows-1252').readlines()]
  texts = [ i[:-1].split('\t' )[1] for i in open(file_name, encoding = 'windows-1252').readlines()]
  aug  = pd.DataFrame({'text':texts, 'labels': labels})
  lbl2indx = {i:j for j, i in enumerate(aug.labels.unique())}
  aug['lbl_index'] = aug.labels.apply(lambda x:lbl2indx[x])
  return aug

In [0]:
aug_data =  read_eda_trec50('trec50_splited/augmented_with_eda/trec-50_clear_5k_samples.txt')

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_5k.csv')

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 564 out of 14248 unique words
Average sentence len:  9.438543954215746
Max sentence len:  37
2.704% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
9.180% of tokens were replaced by unk token index
Train on 30054 samples, validate on 500 samples
Epoch 1/100000
30054/30054 [==============================] - 1s 41us/step - loss: 2.2800 - accuracy: 0.4438 - f1: 0.2995 - val_loss: 1.5262 - val_accuracy: 0.6320 - val_f1: 0.6385
Epoch 2/100000
30054/30054 [==============================] - 1s 33us/step - loss: 0.6988 - accuracy: 0.8346 - f1: 0.8135 - val_loss: 0.9098 - val_accuracy: 0.7740 - val_f1: 0.8027
Epoch 3/100000
30054/30054 [==============================] - 1s 33us/step - loss: 0.1643 - accuracy: 0.9711 - f1: 0.9622 - val_loss: 0.8169 - val_accuracy: 0.8080 - val_f1: 0.8264
Epoch 4/100000
30054/30054 [==============================] - 1s 34us/step - loss: 0.0480 - accuracy: 0.9946 - f1: 0.9924 - val_loss: 0.8112 - val_

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 564 out of 14248 unique words
Average sentence len:  9.438543954215746
Max sentence len:  37
2.704% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
9.180% of tokens were replaced by unk token index
Train on 30054 samples, validate on 500 samples
Epoch 1/100000
30054/30054 [==============================] - 18s 586us/step - loss: 2.9144 - accuracy: 0.2691 - f1: 0.0970 - val_loss: 2.3789 - val_accuracy: 0.4500 - val_f1: 0.1979
Epoch 2/100000
30054/30054 [==============================] - 16s 530us/step - loss: 2.0123 - accuracy: 0.4799 - f1: 0.4584 - val_loss: 1.7416 - val_accuracy: 0.5800 - val_f1: 0.6216
Epoch 3/100000
30054/30054 [==============================] - 16s 531us/step - loss: 1.5051 - accuracy: 0.6067 - f1: 0.5905 - val_loss: 1.4034 - val_accuracy: 0.6360 - val_f1: 0.6931
Epoch 4/100000
30054/30054 [==============================] - 16s 538us/step - loss: 1.0752 - accuracy: 0.7120 - f1: 0.7116 - val_loss: 1.342

### 4000

In [0]:
aug_data =  read_eda_trec50('trec50_splited/augmented_with_eda/trec-50_clear_4k_samples.txt')

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_4k.csv')

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 470 out of 12417 unique words
Average sentence len:  9.427070302727046
Max sentence len:  37
2.730% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
10.165% of tokens were replaced by unk token index
Train on 23982 samples, validate on 500 samples
Epoch 1/100000
23982/23982 [==============================] - 1s 45us/step - loss: 2.6067 - accuracy: 0.3661 - f1: 0.1782 - val_loss: 1.8473 - val_accuracy: 0.5920 - val_f1: 0.5225
Epoch 2/100000
23982/23982 [==============================] - 1s 33us/step - loss: 1.0036 - accuracy: 0.7594 - f1: 0.7134 - val_loss: 0.9547 - val_accuracy: 0.7680 - val_f1: 0.7819
Epoch 3/100000
23982/23982 [==============================] - 1s 33us/step - loss: 0.2613 - accuracy: 0.9516 - f1: 0.9332 - val_loss: 0.7953 - val_accuracy: 0.8140 - val_f1: 0.8165
Epoch 4/100000
23982/23982 [==============================] - 1s 33us/step - loss: 0.0709 - accuracy: 0.9917 - f1: 0.9873 - val_loss: 0.7499 - val

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 470 out of 12417 unique words
Average sentence len:  9.427070302727046
Max sentence len:  37
2.730% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
10.165% of tokens were replaced by unk token index
Train on 23982 samples, validate on 500 samples
Epoch 1/100000
23982/23982 [==============================] - 13s 557us/step - loss: 3.0853 - accuracy: 0.2397 - f1: 0.0604 - val_loss: 2.4413 - val_accuracy: 0.4680 - val_f1: 0.1703
Epoch 2/100000
23982/23982 [==============================] - 13s 529us/step - loss: 2.2538 - accuracy: 0.4295 - f1: 0.3648 - val_loss: 1.9129 - val_accuracy: 0.5540 - val_f1: 0.5572
Epoch 3/100000
23982/23982 [==============================] - 12s 509us/step - loss: 1.7552 - accuracy: 0.5458 - f1: 0.5262 - val_loss: 1.5282 - val_accuracy: 0.6040 - val_f1: 0.6410
Epoch 4/100000
23982/23982 [==============================] - 12s 516us/step - loss: 1.3445 - accuracy: 0.6417 - f1: 0.6352 - val_loss: 1.33

### 3000

In [0]:
aug_data =  read_eda_trec50('trec50_splited/augmented_with_eda/trec-50_clear_3k_samples.txt')

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_3k.csv')

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 319 out of 9331 unique words
Average sentence len:  9.484951603145795
Max sentence len:  33
2.700% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
11.309% of tokens were replaced by unk token index
Train on 13224 samples, validate on 500 samples
Epoch 1/100000
13224/13224 [==============================] - 1s 56us/step - loss: 2.6363 - accuracy: 0.3272 - f1: 0.0669 - val_loss: 2.6541 - val_accuracy: 0.4680 - val_f1: 0.1304
Epoch 2/100000
13224/13224 [==============================] - 0s 32us/step - loss: 1.2494 - accuracy: 0.7172 - f1: 0.6143 - val_loss: 1.8026 - val_accuracy: 0.6440 - val_f1: 0.5927
Epoch 3/100000
13224/13224 [==============================] - 0s 32us/step - loss: 0.5108 - accuracy: 0.8903 - f1: 0.8890 - val_loss: 1.4135 - val_accuracy: 0.6880 - val_f1: 0.7051
Epoch 4/100000
13224/13224 [==============================] - 0s 33us/step - loss: 0.2426 - accuracy: 0.9477 - f1: 0.9490 - val_loss: 1.1730 - val_

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 319 out of 9331 unique words
Average sentence len:  9.484951603145795
Max sentence len:  33
2.700% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
11.309% of tokens were replaced by unk token index
Train on 13224 samples, validate on 500 samples
Epoch 1/100000
13224/13224 [==============================] - 8s 597us/step - loss: 3.0952 - accuracy: 0.2005 - f1: 0.0073 - val_loss: 3.1231 - val_accuracy: 0.2420 - val_f1: 0.0040
Epoch 2/100000
13224/13224 [==============================] - 7s 508us/step - loss: 2.3525 - accuracy: 0.3811 - f1: 0.2350 - val_loss: 2.6493 - val_accuracy: 0.4480 - val_f1: 0.1900
Epoch 3/100000
13224/13224 [==============================] - 7s 531us/step - loss: 1.7411 - accuracy: 0.5258 - f1: 0.4497 - val_loss: 2.2030 - val_accuracy: 0.5660 - val_f1: 0.2466
Epoch 4/100000
13224/13224 [==============================] - 7s 518us/step - loss: 1.2600 - accuracy: 0.6651 - f1: 0.6338 - val_loss: 1.8661 - 

#Trec-6

In [0]:
import pandas as pd
def read_trec6(file_name, typ=0):
  train = pd.read_csv(file_name, index_col=0)
  lbl2indx = {i:j for j, i in enumerate(train.labels.unique())}
  train.columns = ['text', 'labels']
  train['lbl_index'] = train.labels.apply(lambda x:lbl2indx[x])


  labels_t = [i.split(' ', 1)[0].split(':')[typ] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  test  = pd.DataFrame({'text':texts_t, 'labels': labels_t})
  test['lbl_index'] = test.labels.apply(lambda x:lbl2indx[x])

  return train, test, lbl2indx

### 3000

In [0]:
number_of_attemp_to_fit = 3

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_3k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 132 out of 6440 unique words
Average sentence len:  10.175099866844208
Max sentence len:  37
1.937% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
13.199% of tokens were replaced by unk token index
Train on 3004 samples, validate on 500 samples
Epoch 1/100000
3004/3004 [==============================] - 0s 142us/step - loss: 1.6344 - accuracy: 0.3532 - f1: 0.0000e+00 - val_loss: 1.4081 - val_accuracy: 0.5580 - val_f1: 0.0040
Epoch 2/100000
3004/3004 [==============================] - 0s 33us/step - loss: 1.2895 - accuracy: 0.5975 - f1: 0.1410 - val_loss: 1.1649 - val_accuracy: 0.5360 - val_f1: 0.5035
Epoch 3/100000
3004/3004 [==============================] - 0s 33us/step - loss: 0.9520 - accuracy: 0.6947 - f1: 0.5445 - val_loss: 0.9115 - val_accuracy: 0.6580 - val_f1: 0.6118
Epoch 4/100000
3004/3004 [==============================] - 0s 33us/step - loss: 0.6592 - accuracy: 0.8119 - f1: 0.7452 - val_loss: 0.6893 - val_acc

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 132 out of 6440 unique words
Average sentence len:  10.175099866844208
Max sentence len:  37
1.937% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
13.199% of tokens were replaced by unk token index
Train on 3004 samples, validate on 500 samples
Epoch 1/100000
3004/3004 [==============================] - 3s 922us/step - loss: 1.7305 - accuracy: 0.2796 - f1: 0.0000e+00 - val_loss: 1.6661 - val_accuracy: 0.2840 - val_f1: 0.0000e+00
Epoch 2/100000
3004/3004 [==============================] - 2s 532us/step - loss: 1.5446 - accuracy: 0.3848 - f1: 0.0148 - val_loss: 1.4295 - val_accuracy: 0.3440 - val_f1: 0.1789
Epoch 3/100000
3004/3004 [==============================] - 2s 570us/step - loss: 1.3021 - accuracy: 0.4611 - f1: 0.3057 - val_loss: 1.1516 - val_accuracy: 0.5700 - val_f1: 0.3207
Epoch 4/100000
3004/3004 [==============================] - 2s 513us/step - loss: 1.1292 - accuracy: 0.5649 - f1: 0.4198 - val_loss: 1.0504 - 

### 4000

In [0]:
number_of_attemp_to_fit = 3

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_4k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 161 out of 7745 unique words
Average sentence len:  10.211788211788212
Max sentence len:  37
1.917% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
11.336% of tokens were replaced by unk token index
Train on 4004 samples, validate on 500 samples
Epoch 1/100000
4004/4004 [==============================] - 0s 86us/step - loss: 1.6264 - accuracy: 0.3487 - f1: 0.0065 - val_loss: 1.3990 - val_accuracy: 0.5180 - val_f1: 0.0119
Epoch 2/100000
4004/4004 [==============================] - 0s 39us/step - loss: 1.2346 - accuracy: 0.6026 - f1: 0.2161 - val_loss: 1.1107 - val_accuracy: 0.5640 - val_f1: 0.5513
Epoch 3/100000
4004/4004 [==============================] - 0s 34us/step - loss: 0.8906 - accuracy: 0.7065 - f1: 0.5988 - val_loss: 0.8187 - val_accuracy: 0.6620 - val_f1: 0.6207
Epoch 4/100000
4004/4004 [==============================] - 0s 32us/step - loss: 0.5758 - accuracy: 0.8362 - f1: 0.7960 - val_loss: 0.5693 - val_accuracy

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 161 out of 7745 unique words
Average sentence len:  10.211788211788212
Max sentence len:  37
1.917% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
11.336% of tokens were replaced by unk token index
Train on 4004 samples, validate on 500 samples
Epoch 1/100000
4004/4004 [==============================] - 3s 872us/step - loss: 1.7028 - accuracy: 0.2810 - f1: 0.0000e+00 - val_loss: 1.5977 - val_accuracy: 0.3020 - val_f1: 0.0000e+00
Epoch 2/100000
4004/4004 [==============================] - 2s 521us/step - loss: 1.4682 - accuracy: 0.4136 - f1: 0.1028 - val_loss: 1.3399 - val_accuracy: 0.4580 - val_f1: 0.1763
Epoch 3/100000
4004/4004 [==============================] - 2s 576us/step - loss: 1.2257 - accuracy: 0.5125 - f1: 0.3435 - val_loss: 1.1442 - val_accuracy: 0.5680 - val_f1: 0.3473
Epoch 4/100000
4004/4004 [==============================] - 2s 560us/step - loss: 1.0494 - accuracy: 0.5877 - f1: 0.4762 - val_loss: 0.8894 - 

### 5000

In [0]:
number_of_attemp_to_fit = 3

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_5k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 200 out of 8946 unique words
Average sentence len:  10.192369157011585
Max sentence len:  37
1.897% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
10.271% of tokens were replaced by unk token index
Train on 5006 samples, validate on 500 samples
Epoch 1/100000
5006/5006 [==============================] - 4s 735us/step - loss: 1.6847 - accuracy: 0.2529 - f1: 3.8986e-04 - val_loss: 1.5297 - val_accuracy: 0.3720 - val_f1: 0.0000e+00
Epoch 2/100000
5006/5006 [==============================] - 3s 566us/step - loss: 1.3530 - accuracy: 0.4571 - f1: 0.2293 - val_loss: 1.2347 - val_accuracy: 0.4120 - val_f1: 0.3174
Epoch 3/100000
5006/5006 [==============================] - 3s 523us/step - loss: 1.0903 - accuracy: 0.5859 - f1: 0.4473 - val_loss: 0.9591 - val_accuracy: 0.7180 - val_f1: 0.4773
Epoch 4/100000
5006/5006 [==============================] - 3s 534us/step - loss: 0.8903 - accuracy: 0.6754 - f1: 0.6374 - val_loss: 0.8416 - 

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 200 out of 8946 unique words
Average sentence len:  10.192369157011585
Max sentence len:  37
1.897% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
10.271% of tokens were replaced by unk token index
Train on 5006 samples, validate on 500 samples
Epoch 1/100000
5006/5006 [==============================] - 0s 73us/step - loss: 1.5555 - accuracy: 0.3989 - f1: 0.0035 - val_loss: 1.2834 - val_accuracy: 0.5640 - val_f1: 0.0431
Epoch 2/100000
5006/5006 [==============================] - 0s 33us/step - loss: 1.0454 - accuracy: 0.6670 - f1: 0.4315 - val_loss: 0.8592 - val_accuracy: 0.6920 - val_f1: 0.6807
Epoch 3/100000
5006/5006 [==============================] - 0s 32us/step - loss: 0.6166 - accuracy: 0.8116 - f1: 0.7742 - val_loss: 0.5445 - val_accuracy: 0.8000 - val_f1: 0.7920
Epoch 4/100000
5006/5006 [==============================] - 0s 33us/step - loss: 0.3313 - accuracy: 0.9117 - f1: 0.9056 - val_loss: 0.3786 - val_accuracy

## Trec-6 with eda

In [0]:
def read_eda_trec6(file_name):
  "type = 0 - trec6, type = 1 - trec50"
  labels = [i[:-1].split('\t' )[0] for i in open(file_name, encoding = 'windows-1252').readlines()]
  texts = [ i[:-1].split('\t' )[1] for i in open(file_name, encoding = 'windows-1252').readlines()]
  aug  = pd.DataFrame({'text':texts, 'labels': labels})
  lbl2indx = {i:j for j, i in enumerate(aug.labels.unique())}
  aug['lbl_index'] = aug.labels.apply(lambda x:lbl2indx[x])
  return aug

### 5000

In [0]:
aug_data =  read_eda_trec6('trec6_splited/augmented_with_eda/trec-6_clear_5k_samples.txt')

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_5k.csv')

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 557 out of 14285 unique words
Average sentence len:  9.430017312558263
Max sentence len:  37
2.707% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
9.154% of tokens were replaced by unk token index
Train on 30036 samples, validate on 500 samples
Epoch 1/100000
30036/30036 [==============================] - 8s 277us/step - loss: 0.8957 - accuracy: 0.6696 - f1: 0.5463 - val_loss: 0.4522 - val_accuracy: 0.8600 - val_f1: 0.8623
Epoch 2/100000
30036/30036 [==============================] - 1s 48us/step - loss: 0.1513 - accuracy: 0.9619 - f1: 0.9605 - val_loss: 0.3977 - val_accuracy: 0.8720 - val_f1: 0.8744
Epoch 3/100000
30036/30036 [==============================] - 1s 48us/step - loss: 0.0271 - accuracy: 0.9949 - f1: 0.9949 - val_loss: 0.3660 - val_accuracy: 0.8920 - val_f1: 0.8977
Epoch 4/100000
30036/30036 [==============================] - 1s 47us/step - loss: 0.0087 - accuracy: 0.9990 - f1: 0.9989 - val_loss: 0.3652 - val

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 557 out of 14285 unique words
Average sentence len:  9.430017312558263
Max sentence len:  37
2.707% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
9.154% of tokens were replaced by unk token index
Train on 30036 samples, validate on 500 samples
Epoch 1/100000
30036/30036 [==============================] - 18s 605us/step - loss: 1.2271 - accuracy: 0.5115 - f1: 0.3539 - val_loss: 0.7527 - val_accuracy: 0.7420 - val_f1: 0.7239
Epoch 2/100000
30036/30036 [==============================] - 17s 567us/step - loss: 0.6968 - accuracy: 0.7466 - f1: 0.7324 - val_loss: 0.6165 - val_accuracy: 0.7880 - val_f1: 0.7795
Epoch 3/100000
30036/30036 [==============================] - 17s 570us/step - loss: 0.4925 - accuracy: 0.8259 - f1: 0.8239 - val_loss: 0.4738 - val_accuracy: 0.8300 - val_f1: 0.8306
Epoch 4/100000
30036/30036 [==============================] - 17s 567us/step - loss: 0.3554 - accuracy: 0.8790 - f1: 0.8805 - val_loss: 0.437

### 4000

In [0]:
aug_data =  read_eda_trec6('trec6_splited/augmented_with_eda/trec-6_clear_4k_samples.txt')

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_4k.csv')

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 469 out of 12445 unique words
Average sentence len:  9.462204462204463
Max sentence len:  37
2.737% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
10.271% of tokens were replaced by unk token index
Train on 24024 samples, validate on 500 samples
Epoch 1/100000
24024/24024 [==============================] - 1s 56us/step - loss: 1.0876 - accuracy: 0.5952 - f1: 0.4126 - val_loss: 0.6500 - val_accuracy: 0.7520 - val_f1: 0.7392
Epoch 2/100000
24024/24024 [==============================] - 1s 46us/step - loss: 0.2338 - accuracy: 0.9402 - f1: 0.9358 - val_loss: 0.3878 - val_accuracy: 0.8520 - val_f1: 0.8613
Epoch 3/100000
24024/24024 [==============================] - 1s 46us/step - loss: 0.0403 - accuracy: 0.9921 - f1: 0.9921 - val_loss: 0.4251 - val_accuracy: 0.8760 - val_f1: 0.8801
Epoch 4/100000
24024/24024 [==============================] - 1s 46us/step - loss: 0.0133 - accuracy: 0.9979 - f1: 0.9979 - val_loss: 0.4654 - val

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 469 out of 12445 unique words
Average sentence len:  9.462204462204463
Max sentence len:  37
2.737% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
10.271% of tokens were replaced by unk token index
Train on 24024 samples, validate on 500 samples
Epoch 1/100000
24024/24024 [==============================] - 15s 639us/step - loss: 1.2900 - accuracy: 0.4748 - f1: 0.3057 - val_loss: 0.9062 - val_accuracy: 0.6840 - val_f1: 0.6683
Epoch 2/100000
24024/24024 [==============================] - 14s 595us/step - loss: 0.7683 - accuracy: 0.7145 - f1: 0.6938 - val_loss: 0.6422 - val_accuracy: 0.7820 - val_f1: 0.7833
Epoch 3/100000
24024/24024 [==============================] - 14s 600us/step - loss: 0.5560 - accuracy: 0.8033 - f1: 0.7984 - val_loss: 0.5837 - val_accuracy: 0.8080 - val_f1: 0.8096
Epoch 4/100000
24024/24024 [==============================] - 14s 601us/step - loss: 0.3909 - accuracy: 0.8675 - f1: 0.8677 - val_loss: 0.61

### 3000

In [0]:
aug_data =  read_eda_trec6('trec6_splited/augmented_with_eda/trec-6_clear_3k_samples.txt')

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_3k.csv')

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 305 out of 9471 unique words
Average sentence len:  9.435468128237856
Max sentence len:  37
2.695% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
12.267% of tokens were replaced by unk token index
Train on 13319 samples, validate on 500 samples
Epoch 1/100000
13319/13319 [==============================] - 1s 73us/step - loss: 1.1722 - accuracy: 0.5553 - f1: 0.3452 - val_loss: 1.3702 - val_accuracy: 0.4380 - val_f1: 0.4503
Epoch 2/100000
13319/13319 [==============================] - 1s 45us/step - loss: 0.5254 - accuracy: 0.8231 - f1: 0.8109 - val_loss: 0.8298 - val_accuracy: 0.6980 - val_f1: 0.6906
Epoch 3/100000
13319/13319 [==============================] - 1s 45us/step - loss: 0.2125 - accuracy: 0.9395 - f1: 0.9383 - val_loss: 0.6824 - val_accuracy: 0.7600 - val_f1: 0.7371
Epoch 4/100000
13319/13319 [==============================] - 1s 46us/step - loss: 0.0945 - accuracy: 0.9764 - f1: 0.9759 - val_loss: 0.5718 - val_

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([train_data,aug_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 305 out of 9471 unique words
Average sentence len:  9.435468128237856
Max sentence len:  37
2.695% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
12.267% of tokens were replaced by unk token index
Train on 13319 samples, validate on 500 samples
Epoch 1/100000
13319/13319 [==============================] - 9s 673us/step - loss: 1.4096 - accuracy: 0.4228 - f1: 0.1283 - val_loss: 1.7733 - val_accuracy: 0.4080 - val_f1: 0.2050
Epoch 2/100000
13319/13319 [==============================] - 8s 581us/step - loss: 1.0601 - accuracy: 0.5807 - f1: 0.5208 - val_loss: 1.4262 - val_accuracy: 0.4460 - val_f1: 0.4406
Epoch 3/100000
13319/13319 [==============================] - 8s 574us/step - loss: 0.8480 - accuracy: 0.6640 - f1: 0.6434 - val_loss: 1.0167 - val_accuracy: 0.6200 - val_f1: 0.5799
Epoch 4/100000
13319/13319 [==============================] - 8s 582us/step - loss: 0.6979 - accuracy: 0.7316 - f1: 0.7212 - val_loss: 1.0671 - 

#SST-2

### 3000

In [0]:
train_data = pd.read_csv("sst2-splited/sst2_full_rev_3k.csv", index_col=0)
train_data.columns = ['text', 'lbl_index']

In [0]:
test_data = pd.read_csv("sst2-splited/test.csv", sep = '\t')
test_data.columns = ['text', 'lbl_index']

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 50, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 245 out of 9053 unique words
Average sentence len:  18.787333333333333
Max sentence len:  50
1.728% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
10.139% of tokens were replaced by unk token index
Train on 3000 samples, validate on 1821 samples
Epoch 1/100000
3000/3000 [==============================] - 1s 203us/step - loss: 0.7206 - accuracy: 0.5487 - f1: 0.5507 - val_loss: 0.6515 - val_accuracy: 0.7490 - val_f1: 0.7461
Epoch 2/100000
3000/3000 [==============================] - 0s 155us/step - loss: 0.5985 - accuracy: 0.7857 - f1: 0.7880 - val_loss: 0.5945 - val_accuracy: 0.7359 - val_f1: 0.7340
Epoch 3/100000
3000/3000 [==============================] - 0s 154us/step - loss: 0.4898 - accuracy: 0.8473 - f1: 0.8482 - val_loss: 0.5030 - val_accuracy: 0.8045 - val_f1: 0.8013
Epoch 4/100000
3000/3000 [==============================] - 0s 151us/step - loss: 0.3532 - accuracy: 0.8960 - f1: 0.8960 - val_loss: 0.4

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 245 out of 9053 unique words
Average sentence len:  18.787333333333333
Max sentence len:  50
1.728% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
10.139% of tokens were replaced by unk token index
Train on 3000 samples, validate on 1821 samples
Epoch 1/100000
3000/3000 [==============================] - 7s 2ms/step - loss: 0.6806 - accuracy: 0.5330 - f1: 0.5341 - val_loss: 0.6337 - val_accuracy: 0.7194 - val_f1: 0.7217
Epoch 2/100000
3000/3000 [==============================] - 6s 2ms/step - loss: 0.5641 - accuracy: 0.7633 - f1: 0.7630 - val_loss: 0.5025 - val_accuracy: 0.7732 - val_f1: 0.7723
Epoch 3/100000
3000/3000 [==============================] - 6s 2ms/step - loss: 0.4670 - accuracy: 0.7833 - f1: 0.7840 - val_loss: 0.4517 - val_accuracy: 0.7924 - val_f1: 0.7913
Epoch 4/100000
3000/3000 [==============================] - 6s 2ms/step - loss: 0.4142 - accuracy: 0.8043 - f1: 0.8055 - val_loss: 0.4230 - va

### 4000

In [0]:
train_data = pd.read_csv("sst2-splited/sst2_full_rev_4k.csv", index_col=0)
train_data.columns = ['text', 'lbl_index']

In [0]:
test_data = pd.read_csv("sst2-splited/test.csv", sep = '\t')
test_data.columns = ['text', 'lbl_index']

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 65, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 309 out of 10528 unique words
Average sentence len:  18.64375
Max sentence len:  50
1.730% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
8.730% of tokens were replaced by unk token index
Train on 4000 samples, validate on 1821 samples
Epoch 1/100000
4000/4000 [==============================] - 1s 162us/step - loss: 0.6395 - accuracy: 0.6590 - f1: 0.6612 - val_loss: 0.5504 - val_accuracy: 0.8029 - val_f1: 0.8018
Epoch 2/100000
4000/4000 [==============================] - 0s 83us/step - loss: 0.4319 - accuracy: 0.8493 - f1: 0.8486 - val_loss: 0.4321 - val_accuracy: 0.8127 - val_f1: 0.8113
Epoch 3/100000
4000/4000 [==============================] - 0s 80us/step - loss: 0.2631 - accuracy: 0.9068 - f1: 0.9070 - val_loss: 0.3861 - val_accuracy: 0.8259 - val_f1: 0.8258
Epoch 4/100000
4000/4000 [==============================] - 0s 81us/step - loss: 0.1409 - accuracy: 0.9603 - f1: 0.9603 - val_loss: 0.3902 - val_acc

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 309 out of 10528 unique words
Average sentence len:  18.64375
Max sentence len:  50
1.730% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
8.730% of tokens were replaced by unk token index
Train on 4000 samples, validate on 1821 samples
Epoch 1/100000
4000/4000 [==============================] - 5s 1ms/step - loss: 0.6632 - accuracy: 0.6168 - f1: 0.6200 - val_loss: 0.5530 - val_accuracy: 0.7573 - val_f1: 0.7566
Epoch 2/100000
4000/4000 [==============================] - 4s 973us/step - loss: 0.4995 - accuracy: 0.7793 - f1: 0.7803 - val_loss: 0.4393 - val_accuracy: 0.8023 - val_f1: 0.8013
Epoch 3/100000
4000/4000 [==============================] - 4s 982us/step - loss: 0.4129 - accuracy: 0.8240 - f1: 0.8245 - val_loss: 0.4443 - val_accuracy: 0.7946 - val_f1: 0.7948
Epoch 4/100000
4000/4000 [==============================] - 4s 997us/step - loss: 0.3694 - accuracy: 0.8475 - f1: 0.8467 - val_loss: 0.4094 - val_ac

### 5000

In [0]:
train_data = pd.read_csv("sst2-splited/sst2_full_rev_5k.csv", index_col=0)
train_data.columns = ['text', 'lbl_index']

In [0]:
test_data = pd.read_csv("sst2-splited/test.csv", sep = '\t')
test_data.columns = ['text', 'lbl_index']

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 65, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 361 out of 11811 unique words
Average sentence len:  18.5056
Max sentence len:  51
1.735% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
7.824% of tokens were replaced by unk token index
Train on 5000 samples, validate on 1821 samples
Epoch 1/100000
5000/5000 [==============================] - 1s 115us/step - loss: 0.6494 - accuracy: 0.6352 - f1: 0.6383 - val_loss: 0.5622 - val_accuracy: 0.7655 - val_f1: 0.7654
Epoch 2/100000
5000/5000 [==============================] - 0s 79us/step - loss: 0.4434 - accuracy: 0.8368 - f1: 0.8375 - val_loss: 0.4211 - val_accuracy: 0.8182 - val_f1: 0.8162
Epoch 3/100000
5000/5000 [==============================] - 0s 80us/step - loss: 0.2691 - accuracy: 0.8974 - f1: 0.8978 - val_loss: 0.3827 - val_accuracy: 0.8309 - val_f1: 0.8282
Epoch 4/100000
5000/5000 [==============================] - 0s 78us/step - loss: 0.1442 - accuracy: 0.9536 - f1: 0.9536 - val_loss: 0.4052 - val_accu

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 361 out of 11811 unique words
Average sentence len:  18.5056
Max sentence len:  51
1.735% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
7.824% of tokens were replaced by unk token index
Train on 5000 samples, validate on 1821 samples
Epoch 1/100000
5000/5000 [==============================] - 6s 1ms/step - loss: 0.6521 - accuracy: 0.6216 - f1: 0.6247 - val_loss: 0.4939 - val_accuracy: 0.7897 - val_f1: 0.7865
Epoch 2/100000
5000/5000 [==============================] - 5s 976us/step - loss: 0.5075 - accuracy: 0.7662 - f1: 0.7661 - val_loss: 0.4481 - val_accuracy: 0.8105 - val_f1: 0.8102
Epoch 3/100000
5000/5000 [==============================] - 5s 961us/step - loss: 0.4350 - accuracy: 0.8068 - f1: 0.8067 - val_loss: 0.4637 - val_accuracy: 0.7869 - val_f1: 0.7876
Epoch 4/100000
5000/5000 [==============================] - 5s 957us/step - loss: 0.3961 - accuracy: 0.8276 - f1: 0.8276 - val_loss: 0.4169 - val_acc

## EDA with SST

In [0]:
def read_eda_sst2(file_name):
  labels = [i[:-1].split('\t' )[0] for i in open(file_name, encoding = 'windows-1252').readlines()]
  texts = [ i[:-1].split('\t' )[1] for i in open(file_name, encoding = 'windows-1252').readlines()]
  aug  = pd.DataFrame({'text':texts, 'lbl_index': [int(i) for i in labels]})
  return aug

### 3000

In [0]:
aug_data = read_eda_sst2('sst2-splited/augmented_with_eda/sst2_full_rev_3k.txt')

In [0]:
train_data = pd.read_csv("sst2-splited/sst2_full_rev_3k.csv", index_col=0)
train_data.columns = ['text', 'lbl_index']

In [0]:
test_data = pd.read_csv("sst2-splited/test.csv", sep = '\t')
test_data.columns = ['text', 'lbl_index']

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 50, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([aug_data, train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 355 out of 11503 unique words
Average sentence len:  17.972055555555556
Max sentence len:  53
1.069% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
9.180% of tokens were replaced by unk token index
Train on 18000 samples, validate on 1821 samples
Epoch 1/100000
18000/18000 [==============================] - 2s 85us/step - loss: 0.6401 - accuracy: 0.6474 - f1: 0.6527 - val_loss: 0.6084 - val_accuracy: 0.6711 - val_f1: 0.6749
Epoch 2/100000
18000/18000 [==============================] - 1s 58us/step - loss: 0.2054 - accuracy: 0.9514 - f1: 0.9526 - val_loss: 0.6133 - val_accuracy: 0.7364 - val_f1: 0.7380
Epoch 3/100000
18000/18000 [==============================] - 1s 58us/step - loss: 0.0115 - accuracy: 0.9986 - f1: 0.9984 - val_loss: 0.7512 - val_accuracy: 0.7419 - val_f1: 0.7421
Epoch 4/100000
18000/18000 [==============================] - 1s 58us/step - loss: 0.0015 - accuracy: 0.9999 - f1: 0.9999 - val_loss

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([aug_data, train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 355 out of 11503 unique words
Average sentence len:  17.972055555555556
Max sentence len:  53
1.069% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
9.180% of tokens were replaced by unk token index
Train on 18000 samples, validate on 1821 samples
Epoch 1/100000
18000/18000 [==============================] - 14s 791us/step - loss: 0.6597 - accuracy: 0.5952 - f1: 0.5988 - val_loss: 0.6388 - val_accuracy: 0.6282 - val_f1: 0.6309
Epoch 2/100000
18000/18000 [==============================] - 13s 731us/step - loss: 0.5600 - accuracy: 0.7161 - f1: 0.7148 - val_loss: 0.6803 - val_accuracy: 0.6502 - val_f1: 0.6512
Epoch 3/100000
18000/18000 [==============================] - 13s 740us/step - loss: 0.4353 - accuracy: 0.7967 - f1: 0.7956 - val_loss: 0.7127 - val_accuracy: 0.6425 - val_f1: 0.6440
Epoch 4/100000
18000/18000 [==============================] - 13s 742us/step - loss: 0.3193 - accuracy: 0.8610 - f1: 0.8625 - 

### 4000

In [0]:
aug_data = read_eda_sst2('sst2-splited/augmented_with_eda/sst2_full_rev_4k.txt')

In [0]:
train_data = pd.read_csv("sst2-splited/sst2_full_rev_4k.csv", index_col=0)
train_data.columns = ['text', 'lbl_index']

In [0]:
test_data = pd.read_csv("sst2-splited/test.csv", sep = '\t')
test_data.columns = ['text', 'lbl_index']

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 50, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([aug_data, train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 431 out of 13272 unique words
Average sentence len:  17.875125
Max sentence len:  57
1.060% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
7.926% of tokens were replaced by unk token index
Train on 24000 samples, validate on 1821 samples
Epoch 1/100000
24000/24000 [==============================] - 2s 66us/step - loss: 0.6071 - accuracy: 0.6803 - f1: 0.6808 - val_loss: 0.5681 - val_accuracy: 0.7068 - val_f1: 0.7109
Epoch 2/100000
24000/24000 [==============================] - 1s 57us/step - loss: 0.1271 - accuracy: 0.9697 - f1: 0.9698 - val_loss: 0.6031 - val_accuracy: 0.7529 - val_f1: 0.7542
Epoch 3/100000
24000/24000 [==============================] - 1s 58us/step - loss: 0.0061 - accuracy: 0.9992 - f1: 0.9993 - val_loss: 0.7227 - val_accuracy: 0.7573 - val_f1: 0.7597
Epoch 4/100000
24000/24000 [==============================] - 1s 57us/step - loss: 9.9055e-04 - accuracy: 0.9999 - f1: 0.9999 - val_loss: 0.7

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([aug_data, train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 431 out of 13272 unique words
Average sentence len:  17.875125
Max sentence len:  57
1.060% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
7.926% of tokens were replaced by unk token index
Train on 24000 samples, validate on 1821 samples
Epoch 1/100000
24000/24000 [==============================] - 18s 742us/step - loss: 0.6474 - accuracy: 0.6159 - f1: 0.6160 - val_loss: 0.6530 - val_accuracy: 0.6293 - val_f1: 0.6296
Epoch 2/100000
24000/24000 [==============================] - 17s 711us/step - loss: 0.5503 - accuracy: 0.7208 - f1: 0.7210 - val_loss: 0.6958 - val_accuracy: 0.6452 - val_f1: 0.6472
Epoch 3/100000
24000/24000 [==============================] - 17s 711us/step - loss: 0.4115 - accuracy: 0.8091 - f1: 0.8093 - val_loss: 0.8843 - val_accuracy: 0.6458 - val_f1: 0.6446
Epoch 4/100000
24000/24000 [==============================] - 17s 708us/step - loss: 0.2599 - accuracy: 0.8900 - f1: 0.8901 - val_loss:

### 5000

In [0]:
aug_data = read_eda_sst2('sst2-splited/augmented_with_eda/sst2_full_rev_5k.txt')

In [0]:
train_data = pd.read_csv("sst2-splited/sst2_full_rev_5k.csv", index_col=0)
train_data.columns = ['text', 'lbl_index']

In [0]:
test_data = pd.read_csv("sst2-splited/test.csv", sep = '\t')
test_data.columns = ['text', 'lbl_index']

In [0]:
number_of_attemp_to_fit = 3

In [0]:
MAX_SEQ_LEN, N_CLASSES = 50, len(train_data.lbl_index.unique())

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_cnn(pd.concat([aug_data, train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 514 out of 14580 unique words
Average sentence len:  17.726650726503586
Max sentence len:  54
1.077% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
7.153% of tokens were replaced by unk token index
Train on 27185 samples, validate on 1821 samples
Epoch 1/100000
27185/27185 [==============================] - 2s 72us/step - loss: 0.5781 - accuracy: 0.7045 - f1: 0.7074 - val_loss: 0.5295 - val_accuracy: 0.7287 - val_f1: 0.7281
Epoch 2/100000
27185/27185 [==============================] - 2s 59us/step - loss: 0.1078 - accuracy: 0.9724 - f1: 0.9725 - val_loss: 0.5688 - val_accuracy: 0.7738 - val_f1: 0.7739
Epoch 3/100000
27185/27185 [==============================] - 2s 59us/step - loss: 0.0094 - accuracy: 0.9989 - f1: 0.9989 - val_loss: 0.6436 - val_accuracy: 0.7792 - val_f1: 0.7812
Epoch 4/100000
27185/27185 [==============================] - 2s 58us/step - loss: 0.0015 - accuracy: 0.9999 - f1: 0.9999 - val_loss

In [0]:
f1_scores, acc_scores = [], []
for _ in range(number_of_attemp_to_fit):
  f1_score, acc_score = run_lstm(pd.concat([aug_data, train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )
  f1_scores.append(f1_score)
  acc_scores.append(acc_score)
print("f1_average: {}, std:{}".format(np.mean(f1_scores), np.std(f1_scores)))
print("acc_average: {}, std:{}".format(np.mean(acc_scores), np.std(acc_scores)))

Not found vectors 514 out of 14580 unique words
Average sentence len:  17.726650726503586
Max sentence len:  54
1.077% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
7.153% of tokens were replaced by unk token index
Train on 27185 samples, validate on 1821 samples
Epoch 1/100000
27185/27185 [==============================] - 21s 761us/step - loss: 0.6500 - accuracy: 0.6097 - f1: 0.6111 - val_loss: 0.7131 - val_accuracy: 0.5947 - val_f1: 0.5922
Epoch 2/100000
27185/27185 [==============================] - 19s 716us/step - loss: 0.5574 - accuracy: 0.7092 - f1: 0.7106 - val_loss: 0.6619 - val_accuracy: 0.6535 - val_f1: 0.6569
Epoch 3/100000
27185/27185 [==============================] - 20s 732us/step - loss: 0.4343 - accuracy: 0.7925 - f1: 0.7935 - val_loss: 0.7504 - val_accuracy: 0.6727 - val_f1: 0.6744
Epoch 4/100000
27185/27185 [==============================] - 20s 731us/step - loss: 0.2744 - accuracy: 0.8830 - f1: 0.8839 - 

OLD EDA code without batching embeding vectors

In [0]:
#getting the x and y inputs in numpy array form from the text file
def get_x_y(train_data, num_classes, word2vec, word2vec_len=300, input_size=64):
  num_lines = train_data.shape[0]

  #initialize x and y matrix
  x_matrix = None
  y_matrix = None

  try:
    x_matrix = np.zeros((num_lines, input_size, word2vec_len))
  except:
    print("Error!", num_lines, input_size, word2vec_len)
  y_matrix = np.zeros((num_lines, num_classes))
  s,d = 0,0
  #insert values
  for i, line in enumerate(train_data.iterrows()):
    label = line[1]['lbl_index']
    sentence = line[1]['texts']

    #insert x
    words = sentence.split(' ')
    words = words[:x_matrix.shape[1]] #cut off if too long
    for j, word in enumerate(words):
      if word in word2vec:
        x_matrix[i, j, :] = word2vec[word]
        d += 1
      else:
        s += 1

    #insert y
    y_matrix[i][label] = 1.0
  print('Not found vectors {} out of {} words in dateset'.format(s, d))
  return x_matrix, y_matrix

In [0]:
###################################################
##################### model #######################
###################################################

#building the model in keras
def build_model(sentence_length, word2vec_len, num_classes):
	model = None
	model = Sequential()
 
	model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(sentence_length, word2vec_len)))
	model.add(Dropout(0.5))
	model.add(Bidirectional(LSTM(128, return_sequences=False)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1 ])
	#print(model.summary())
	return model

#building the cnn in keras
def build_cnn(sentence_length, word2vec_matrix, max_seq_len, num_classes):
	model = None
	model = Sequential()
  model.add(Embedding(word2vec_matrix.shape[0], word2vec_matrix[1], input_length=max_seq_len,  weights=[word2vec_matrix]))
	model.add(layers.Conv1D(256, 5, activation='relu', input_shape=(sentence_length, word2vec_len)))
	model.add(layers.GlobalMaxPooling1D())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1])
	return model

In [0]:
def run_model(train_data, test_data, num_classes, word2vec, input_size=64, word2vec_len=300):

  #initialize model
  model = build_model(input_size, word2vec_len, num_classes)

  #load data
  train_x, train_y = get_x_y(train_data, num_classes, w2v_model)
  test_x, test_y = get_x_y(test_data, num_classes, w2v_model)

  #implement early stopping
  callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

  #train model
  model.fit(train_x, 
        train_y, 
        epochs=100000, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=512, 
        shuffle=True, 
        verbose=1)
  #model.save('checkpoints/lol')
  #model = load_model('checkpoints/lol')

  #evaluate model
  y_pred = model.predict(test_x)
  test_y_cat = one_hot_to_categorical(test_y)
  y_pred_cat = one_hot_to_categorical(y_pred)
  f1 = f1_score(test_y_cat, y_pred_cat, average='micro')
  acc = accuracy_score(test_y_cat, y_pred_cat)

  print('Best F1:{}, best accuracy:{}'.format(acc, f1))
  return acc

In [0]:
def run_cnn(train_data, test_data, num_classes, word2vec, input_size =64, word2vec_len=300):

  #initialize model
  model = build_cnn(input_size, word2vec_len, num_classes)

  #load data
  train_x, train_y = get_x_y(train_data, num_classes, w2v_model)
  test_x, test_y = get_x_y(test_data, num_classes, w2v_model)

  #implement early stopping
  callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

  #train model
  model.fit(	train_x, 
        train_y, 
        epochs=100000, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=1024, 
        shuffle=True, 
        verbose=1)
  #model.save('checkpoints/lol')
  #model = load_model('checkpoints/lol')

  #evaluate model
  
  y_pred = model.predict(test_x)
  test_y_cat = one_hot_to_categorical(test_y)
  y_pred_cat = one_hot_to_categorical(y_pred)
  f1 = f1_score(test_y_cat, y_pred_cat, average='micro')
  acc = accuracy_score(test_y_cat, y_pred_cat)
  
  print('Best F1:{}, best accuracy:{}'.format(acc, f1))

  #clean memory???
  train_x, train_y, model = None, None, None
  gc.collect()

  #return the accuracy
  #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
  return acc

In [0]:
import os

import tensorflow as tf
import tensorflow_datasets

from transformers import (
    BertConfig,
    BertForSequenceClassification,
    BertTokenizer,
    TFBertForSequenceClassification,
    glue_convert_examples_to_features,
    glue_processors,
)


# script parameters
BATCH_SIZE = 32
EVAL_BATCH_SIZE = BATCH_SIZE * 2

EPOCHS = 5

TASK = "sst-2"
num_labels = 2

tf.config.optimizer.set_jit(False)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": False})

# Load tokenizer and model from pretrained model/vocabulary. Specify the number of labels to classify (2+: classification, 1: regression)
config = BertConfig.from_pretrained("bert-base-cased", num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-cased", config=config) #build_cnn()


# Load dataset via TensorFlow Datasets
data, info = tensorflow_datasets.load(f"glue/sst2", with_info=True)
train_examples = 6000#info.splits["train"].num_examples
print(train_examples)

# MNLI expects either validation_matched or validation_mismatched
valid_examples = info.splits["validation"].num_examples
print(info.splits["validation"].num_examples)

# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data["train"], tokenizer, max_length=128, task=TASK)

# MNLI expects either validation_matched or validation_mismatched
valid_dataset = glue_convert_examples_to_features(data["validation"], tokenizer, max_length=128, task=TASK)
train_dataset = train_dataset.shuffle(128).take(train_examples).batch(BATCH_SIZE).repeat(-1)
valid_dataset = valid_dataset.batch(EVAL_BATCH_SIZE)

# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)


loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
model.compile(optimizer=opt, loss=loss, metrics=[metric, f1])

# Train and evaluate using tf.keras.Model.fit()
train_steps = train_examples // BATCH_SIZE
valid_steps = valid_examples // EVAL_BATCH_SIZE

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_data=valid_dataset,
    validation_steps=valid_steps,
)

INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for glue/sst2/1.0.0
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/sst2/1.0.0)


INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8 into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.cowOhVrpNUsvqdZqI70Nq3ISu63l9SOhTqYqoz6uEW3-Y.zipalt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8.tmp.f383cd306f334141ab36fd7d0f8787c6...
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/glue/sst2/1.0.0.incompleteP8IX4E/glue-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/sst2/1.0.0.incompleteP8IX4E/glue-train.tfrecord. Shard lengths: [67349]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/glue/sst2/1.0.0.incompleteP8IX4E/glue-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/sst2/1.0.0.incompleteP8IX4E/glue-validation.tfrecord. Shard lengths: [872]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/glue/sst2/1.0.0.incompleteP8IX4E/glue-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/sst2/1.0.0.incompleteP8IX4E/glue-test.tfrecord. Shard lengths: [1821]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/glue/sst2/1.0.0


Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/sst2/1.0.0. Subsequent calls will reuse this data.
6000
872
Epoch 1/5
187/187 [==============================] - 199s 1s/step - loss: 0.3466 - accuracy: 0.8509 - f1: 0.6644 - val_loss: 0.3620 - val_accuracy: 0.8474 - val_f1: 0.7192
Epoch 2/5
187/187 [==============================] - 198s 1s/step - loss: 0.1543 - accuracy: 0.9452 - f1: 0.7082 - val_loss: 0.3197 - val_accuracy: 0.8858 - val_f1: 0.6761
Epoch 3/5
187/187 [==============================] - 196s 1s/step - loss: 0.0734 - accuracy: 0.9762 - f1: 0.7150 - val_loss: 0.3237 - val_accuracy: 0.8918 - val_f1: 0.6659
Epoch 4/5
187/187 [==============================] - 196s 1s/step - loss: 0.0654 - accuracy: 0.9787 - f1: 0.7133 - val_loss: 0.4249 - val_accuracy: 0.8858 - val_f1: 0.6710
Epoch 5/5
187/187 [==============================] - 196s 1s/step - loss: 0.0353 - accuracy: 0.9891 - f1: 0.7133 - val_loss: 0.4886 - val_accuracy: 0.8882 - val_f1: 0.6751


# SST2 aug bert 

In [0]:
import pandas as pd
def read_sst(file_name):
  i2l = {1:"pos", 0:"neg"}
  train_data = pd.read_csv(file_name, index_col=0)
  train_data.columns = ['text', 'lbl_index']
  train_data["labels"] = train_data.lbl_index.apply(lambda x:i2l[x])
  test_data = pd.read_csv("sst2-splited/test.csv", sep = '\t')
  test_data.columns = ['text', 'lbl_index']
  test_data["labels"] = test_data.lbl_index.apply(lambda x:i2l[x])
  return train_data, test_data, {i:j for j,i in i2l.items()}

## 3k

In [0]:
!ls

In [0]:
train_data, test_data, l2i = read_sst('sst2-splited/sst2_full_rev_3k.csv')

In [0]:
from glob import glob 
files = glob('./sst2-splited/sst_Splited_3k_bert_*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
MAX_SEQ_LEN, N_CLASSES = 50, len(train_data.lbl_index.unique())

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(pd.concat([all_augs[:500000], train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 266487 out of 392639 unique words
Average sentence len:  19.741760992003904
Max sentence len:  79
8.959% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.606% of tokens were replaced by unk token index
Train on 475482 samples, validate on 1821 samples
Epoch 1/1
475482/475482 [==============================] - 288s 606us/step - loss: 0.5164 - accuracy: 0.7399 - f1: 0.7399 - val_loss: 0.4367 - val_accuracy: 0.7957 - val_f1: 0.7939
Best F1:0.794, best accuracy:0.796


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  18.787333333333333
Max sentence len:  50
1.728% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.606% of tokens were replaced by unk token index
Train on 3000 samples, validate on 1821 samples
Epoch 1/100
3000/3000 [==============================] - 2s 662us/step - loss: 0.4122 - accuracy: 0.8023 - f1: 0.8027 - val_loss: 0.4180 - val_accuracy: 0.8012 - val_f1: 0.8003
Epoch 2/100
3000/3000 [==============================] - 2s 670us/step - loss: 0.3711 - accuracy: 0.8293 - f1: 0.8289 - val_loss: 0.4264 - val_accuracy: 0.8051 - val_f1: 0.8037
Epoch 3/100
3000/3000 [==============================] - 2s 682us/step - loss: 0.3214 - accuracy: 0.8623 - f1: 0.8625 - val_loss: 0.4242 - val_accuracy: 0.8160 - val_f1: 0.8119
Epoch 4/100
3000/3000 [==============================] - 2s 656us/step - loss: 0.2651 - accuracy: 0.8887 - f1: 0.8886 - val_loss: 0.4522 - val_accuracy: 0.8111 - val_f1: 0.8099
Epoch 5/100
3000

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(pd.concat([all_augs[:500000], train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 266487 out of 392639 unique words
Average sentence len:  19.741760992003904
Max sentence len:  79
8.959% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.606% of tokens were replaced by unk token index
Train on 475482 samples, validate on 1821 samples
Epoch 1/1
475482/475482 [==============================] - 28s 59us/step - loss: 0.4303 - accuracy: 0.7943 - f1: 0.7943 - val_loss: 0.3501 - val_accuracy: 0.8369 - val_f1: 0.8379
Best F1:0.838, best accuracy:0.837


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  18.787333333333333
Max sentence len:  50
1.728% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.606% of tokens were replaced by unk token index
Train on 3000 samples, validate on 1821 samples
Epoch 1/100
3000/3000 [==============================] - 0s 68us/step - loss: 0.2293 - accuracy: 0.9107 - f1: 0.9105 - val_loss: 0.3375 - val_accuracy: 0.8418 - val_f1: 0.8415
Epoch 2/100
3000/3000 [==============================] - 0s 55us/step - loss: 0.1571 - accuracy: 0.9447 - f1: 0.9447 - val_loss: 0.3566 - val_accuracy: 0.8484 - val_f1: 0.8477
Epoch 3/100
3000/3000 [==============================] - 0s 56us/step - loss: 0.0820 - accuracy: 0.9800 - f1: 0.9803 - val_loss: 0.3772 - val_accuracy: 0.8550 - val_f1: 0.8551
Epoch 4/100
3000/3000 [==============================] - 0s 55us/step - loss: 0.0341 - accuracy: 0.9953 - f1: 0.9954 - val_loss: 0.3999 - val_accuracy: 0.8528 - val_f1: 0.8532
Epoch 5/100
3000/300

## 4k

In [0]:
train_data, test_data, l2i = read_sst('sst2-splited/sst2_full_rev_4k.csv')

In [0]:
from glob import glob 
files = glob('./sst2-splited/sst_Splited_4k_bert_*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
MAX_SEQ_LEN, N_CLASSES = 50, len(train_data.lbl_index.unique())

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(pd.concat([all_augs[:500000], train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 267107 out of 393569 unique words
Average sentence len:  19.725626621384794
Max sentence len:  79
8.950% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.529% of tokens were replaced by unk token index
Train on 477601 samples, validate on 1821 samples
Epoch 1/1
477601/477601 [==============================] - 295s 618us/step - loss: 0.5064 - accuracy: 0.7458 - f1: 0.7458 - val_loss: 0.4314 - val_accuracy: 0.7979 - val_f1: 0.7981
Best F1:0.798, best accuracy:0.798


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  18.64375
Max sentence len:  50
1.730% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.529% of tokens were replaced by unk token index
Train on 4000 samples, validate on 1821 samples
Epoch 1/100
4000/4000 [==============================] - 3s 665us/step - loss: 0.3986 - accuracy: 0.8185 - f1: 0.8184 - val_loss: 0.4313 - val_accuracy: 0.7952 - val_f1: 0.7938
Epoch 2/100
4000/4000 [==============================] - 3s 662us/step - loss: 0.3513 - accuracy: 0.8438 - f1: 0.8440 - val_loss: 0.4332 - val_accuracy: 0.8034 - val_f1: 0.8015
Epoch 3/100
4000/4000 [==============================] - 3s 671us/step - loss: 0.3105 - accuracy: 0.8673 - f1: 0.8673 - val_loss: 0.4217 - val_accuracy: 0.8243 - val_f1: 0.8212
Epoch 4/100
4000/4000 [==============================] - 3s 676us/step - loss: 0.2451 - accuracy: 0.8992 - f1: 0.9003 - val_loss: 0.4672 - val_accuracy: 0.8248 - val_f1: 0.8252
Epoch 5/100
4000/4000 [===

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(pd.concat([all_augs[:500000], train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 267107 out of 393569 unique words
Average sentence len:  19.725626621384794
Max sentence len:  79
8.950% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.529% of tokens were replaced by unk token index
Train on 477601 samples, validate on 1821 samples
Epoch 1/1
477601/477601 [==============================] - 24s 50us/step - loss: 0.4204 - accuracy: 0.7983 - f1: 0.7983 - val_loss: 0.3430 - val_accuracy: 0.8435 - val_f1: 0.8437
Best F1:0.844, best accuracy:0.843


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  18.64375
Max sentence len:  50
1.730% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.529% of tokens were replaced by unk token index
Train on 4000 samples, validate on 1821 samples
Epoch 1/100
4000/4000 [==============================] - 0s 63us/step - loss: 0.2226 - accuracy: 0.9180 - f1: 0.9182 - val_loss: 0.3461 - val_accuracy: 0.8462 - val_f1: 0.8477
Epoch 2/100
4000/4000 [==============================] - 0s 57us/step - loss: 0.1458 - accuracy: 0.9530 - f1: 0.9530 - val_loss: 0.3677 - val_accuracy: 0.8468 - val_f1: 0.8470
Epoch 3/100
4000/4000 [==============================] - 0s 55us/step - loss: 0.0694 - accuracy: 0.9847 - f1: 0.9846 - val_loss: 0.4022 - val_accuracy: 0.8440 - val_f1: 0.8446
Epoch 4/100
4000/4000 [==============================] - 0s 55us/step - loss: 0.0294 - accuracy: 0.9942 - f1: 0.9943 - val_loss: 0.4523 - val_accuracy: 0.8424 - val_f1: 0.8428
Epoch 5/100
4000/4000 [=======

## 5k

In [0]:
train_data, test_data, l2i = read_sst('sst2-splited/sst2_full_rev_5k.csv')

In [0]:
from glob import glob 
files = glob('./sst2-splited/sst_Splited_4k_bert_*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
MAX_SEQ_LEN, N_CLASSES = 50, len(train_data.lbl_index.unique())

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(pd.concat([all_augs[:500000], train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 267156 out of 393779 unique words
Average sentence len:  19.72192285431915
Max sentence len:  79
8.937% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.449% of tokens were replaced by unk token index
Train on 478601 samples, validate on 1821 samples
Epoch 1/1
478601/478601 [==============================] - 291s 608us/step - loss: 0.5105 - accuracy: 0.7429 - f1: 0.7429 - val_loss: 0.4264 - val_accuracy: 0.8105 - val_f1: 0.8113
Best F1:0.811, best accuracy:0.811


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  18.5056
Max sentence len:  51
1.735% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.449% of tokens were replaced by unk token index
Train on 5000 samples, validate on 1821 samples
Epoch 1/100
5000/5000 [==============================] - 3s 636us/step - loss: 0.4093 - accuracy: 0.8080 - f1: 0.8080 - val_loss: 0.4270 - val_accuracy: 0.8001 - val_f1: 0.7997
Epoch 2/100
5000/5000 [==============================] - 3s 652us/step - loss: 0.3552 - accuracy: 0.8386 - f1: 0.8385 - val_loss: 0.4048 - val_accuracy: 0.8193 - val_f1: 0.8172
Epoch 3/100
5000/5000 [==============================] - 3s 654us/step - loss: 0.3028 - accuracy: 0.8710 - f1: 0.8711 - val_loss: 0.4201 - val_accuracy: 0.8243 - val_f1: 0.8227
Epoch 4/100
5000/5000 [==============================] - 3s 643us/step - loss: 0.2485 - accuracy: 0.8990 - f1: 0.8990 - val_loss: 0.4811 - val_accuracy: 0.8007 - val_f1: 0.7975
Epoch 5/100
5000/5000 [====

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(pd.concat([all_augs[:500000], train_data]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 267156 out of 393779 unique words
Average sentence len:  19.72192285431915
Max sentence len:  79
8.937% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.449% of tokens were replaced by unk token index
Train on 478601 samples, validate on 1821 samples
Epoch 1/1
478601/478601 [==============================] - 23s 49us/step - loss: 0.4187 - accuracy: 0.7990 - f1: 0.7990 - val_loss: 0.3624 - val_accuracy: 0.8347 - val_f1: 0.8347
Best F1:0.835, best accuracy:0.835


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  18.5056
Max sentence len:  51
1.735% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
2.449% of tokens were replaced by unk token index
Train on 5000 samples, validate on 1821 samples
Epoch 1/100
5000/5000 [==============================] - 0s 60us/step - loss: 0.2365 - accuracy: 0.9056 - f1: 0.9050 - val_loss: 0.3766 - val_accuracy: 0.8391 - val_f1: 0.8387
Epoch 2/100
5000/5000 [==============================] - 0s 55us/step - loss: 0.1336 - accuracy: 0.9570 - f1: 0.9568 - val_loss: 0.3926 - val_accuracy: 0.8468 - val_f1: 0.8451
Epoch 3/100
5000/5000 [==============================] - 0s 54us/step - loss: 0.0588 - accuracy: 0.9870 - f1: 0.9871 - val_loss: 0.4132 - val_accuracy: 0.8468 - val_f1: 0.8447
Epoch 4/100
5000/5000 [==============================] - 0s 54us/step - loss: 0.0213 - accuracy: 0.9968 - f1: 0.9968 - val_loss: 0.4669 - val_accuracy: 0.8479 - val_f1: 0.8457
Epoch 5/100
5000/5000 [========

# TREC-6 aug bert

In [0]:
import pandas as pd
def read_trec6(file_name, typ=0):
  train = pd.read_csv(file_name, index_col=0)
  lbl2indx = {i:j for j, i in enumerate(train.labels.unique())}
  train.columns = ['text', 'labels']
  train['lbl_index'] = train.labels.apply(lambda x:lbl2indx[x])


  labels_t = [i.split(' ', 1)[0].split(':')[typ] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  test  = pd.DataFrame({'text':texts_t, 'labels': labels_t})
  test['lbl_index'] = test.labels.apply(lambda x:lbl2indx[x])

  return train, test, lbl2indx
  

In [0]:
from keras.callbacks import History
def re_fit_lstm(train_data, test_data, w2v_model, msl, n_c, model=None, epochs=100, word2ind_aug=None):

  #load data
  word2ind=None
  if word2ind_aug is None:
    train_x, train_y, word2ind, in2word, pretrained_vectors = create_train_index(train_data, w2v_model, msl, n_c)
    test_x, test_y = transform(test_data, word2ind, n_c, msl)
  else:
    train_x, train_y = transform(train_data, word2ind_aug, n_c, msl)
    test_x, test_y = transform(test_data, word2ind_aug, n_c, msl)

  #initialize model
  if model is None:
    model = build_lstm(pretrained_vectors, msl, n_c)

  #implement early stopping
  history = History()
  callbacks = [EarlyStopping(monitor='val_loss', patience=5), history]

  #train model
  hist = model.fit(train_x, 
        train_y, 
        epochs=epochs, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=512, 
        shuffle=True, 
        verbose=1)
  all_f1 = hist.history['val_f1']
  all_acc = hist.history['val_accuracy']

  print('Best F1:{0:0.3f}, best accuracy:{1:0.3f}'.format(max(all_f1), max(all_acc)))
  return max(all_f1), max(all_acc), model, word2ind

In [0]:
def re_fit_cnn(train_data, test_data, w2v_model, msl, n_c, model=None, epochs=100, word2ind_aug=None):

  #load data
  word2ind=None
  if word2ind_aug is None:
    train_x, train_y, word2ind, in2word, pretrained_vectors = create_train_index(train_data, w2v_model, msl, n_c)
    test_x, test_y = transform(test_data, word2ind, n_c, msl)
  else:
    train_x, train_y = transform(train_data, word2ind_aug, n_c, msl)
    test_x, test_y = transform(test_data, word2ind_aug, n_c, msl)

  #initialize model
  if model is None:
    model = build_cnn(pretrained_vectors, msl, n_c)

  #implement early stopping
  history = History()
  callbacks = [EarlyStopping(monitor='val_loss', patience=5), history]

  #train model
  hist = model.fit(train_x, 
        train_y, 
        epochs=epochs, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=512, 
        shuffle=True, 
        verbose=1)
  all_f1 = hist.history['val_f1']
  all_acc = hist.history['val_accuracy']

  print('Best F1:{0:0.3f}, best accuracy:{1:0.3f}'.format(max(all_f1), max(all_acc)))
  return max(all_f1), max(all_acc), model, word2ind

## 3k

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_3k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./Question_augmentation/TREC6_3k_bert_0.7__*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 39860 out of 108255 unique words
Average sentence len:  9.719566666666667
Max sentence len:  25601
5.920% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.353% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 273s 912us/step - loss: 0.4846 - accuracy: 0.8230 - f1: 0.8090 - val_loss: 0.3982 - val_accuracy: 0.8520 - val_f1: 0.8457
Best F1:0.846, best accuracy:0.852


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.175099866844208
Max sentence len:  37
6.517% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.353% of tokens were replaced by unk token index
Train on 3004 samples, validate on 500 samples
Epoch 1/100
3004/3004 [==============================] - 3s 963us/step - loss: 0.3856 - accuracy: 0.8715 - f1: 0.8692 - val_loss: 0.4496 - val_accuracy: 0.8060 - val_f1: 0.7853
Epoch 2/100
3004/3004 [==============================] - 3s 974us/step - loss: 0.3374 - accuracy: 0.8878 - f1: 0.8834 - val_loss: 0.2936 - val_accuracy: 0.9040 - val_f1: 0.9119
Epoch 3/100
3004/3004 [==============================] - 3s 957us/step - loss: 0.2902 - accuracy: 0.9005 - f1: 0.9024 - val_loss: 0.2840 - val_accuracy: 0.9140 - val_f1: 0.9109
Epoch 4/100
3004/3004 [==============================] - 3s 951us/step - loss: 0.2453 - accuracy: 0.9144 - f1: 0.9170 - val_loss: 0.2688 - val_accuracy: 0.9140 - val_f1: 0.9133
Epoch 5/100
3004/3004 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 39860 out of 108255 unique words
Average sentence len:  9.719566666666667
Max sentence len:  25601
5.920% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.353% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 32s 107us/step - loss: 0.3634 - accuracy: 0.8660 - f1: 0.8557 - val_loss: 0.2180 - val_accuracy: 0.9360 - val_f1: 0.9371
Best F1:0.937, best accuracy:0.936


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.175099866844208
Max sentence len:  37
6.517% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.353% of tokens were replaced by unk token index
Train on 3004 samples, validate on 500 samples
Epoch 1/100
3004/3004 [==============================] - 1s 167us/step - loss: 0.2561 - accuracy: 0.9111 - f1: 0.9091 - val_loss: 0.2114 - val_accuracy: 0.9400 - val_f1: 0.9431
Epoch 2/100
3004/3004 [==============================] - 0s 102us/step - loss: 0.1597 - accuracy: 0.9517 - f1: 0.9513 - val_loss: 0.1523 - val_accuracy: 0.9580 - val_f1: 0.9544
Epoch 3/100
3004/3004 [==============================] - 0s 96us/step - loss: 0.0941 - accuracy: 0.9780 - f1: 0.9781 - val_loss: 0.1590 - val_accuracy: 0.9540 - val_f1: 0.9577
Epoch 4/100
3004/3004 [==============================] - 0s 97us/step - loss: 0.0557 - accuracy: 0.9903 - f1: 0.9901 - val_loss: 0.1334 - val_accuracy: 0.9600 - val_f1: 0.9587
Epoch 5/100
3004/3004 [=========

## 4k

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_4k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./Question_augmentation/TREC6_4k_bert_0.7__*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 39948 out of 108102 unique words
Average sentence len:  9.71283
Max sentence len:  25601
5.950% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.353% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 275s 918us/step - loss: 0.4746 - accuracy: 0.8265 - f1: 0.8146 - val_loss: 0.3884 - val_accuracy: 0.8520 - val_f1: 0.8462
Best F1:0.846, best accuracy:0.852


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.211788211788212
Max sentence len:  37
6.283% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.353% of tokens were replaced by unk token index
Train on 4004 samples, validate on 500 samples
Epoch 1/100
4004/4004 [==============================] - 4s 957us/step - loss: 0.3641 - accuracy: 0.8734 - f1: 0.8728 - val_loss: 0.2905 - val_accuracy: 0.9080 - val_f1: 0.9058
Epoch 2/100
4004/4004 [==============================] - 4s 952us/step - loss: 0.3077 - accuracy: 0.8996 - f1: 0.8985 - val_loss: 0.2520 - val_accuracy: 0.9120 - val_f1: 0.9082
Epoch 3/100
4004/4004 [==============================] - 4s 944us/step - loss: 0.2592 - accuracy: 0.9163 - f1: 0.9182 - val_loss: 0.2422 - val_accuracy: 0.9120 - val_f1: 0.9153
Epoch 4/100
4004/4004 [==============================] - 4s 956us/step - loss: 0.2093 - accuracy: 0.9301 - f1: 0.9312 - val_loss: 0.2344 - val_accuracy: 0.9160 - val_f1: 0.9188
Epoch 5/100
4004/4004 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 39948 out of 108102 unique words
Average sentence len:  9.71283
Max sentence len:  25601
5.950% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.353% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 27s 91us/step - loss: 0.3524 - accuracy: 0.8714 - f1: 0.8609 - val_loss: 0.2392 - val_accuracy: 0.9360 - val_f1: 0.9275
Best F1:0.927, best accuracy:0.936


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.211788211788212
Max sentence len:  37
6.283% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.353% of tokens were replaced by unk token index
Train on 4004 samples, validate on 500 samples
Epoch 1/100
4004/4004 [==============================] - 0s 99us/step - loss: 0.2206 - accuracy: 0.9251 - f1: 0.9256 - val_loss: 0.1536 - val_accuracy: 0.9560 - val_f1: 0.9557
Epoch 2/100
4004/4004 [==============================] - 0s 95us/step - loss: 0.1223 - accuracy: 0.9673 - f1: 0.9661 - val_loss: 0.1434 - val_accuracy: 0.9580 - val_f1: 0.9546
Epoch 3/100
4004/4004 [==============================] - 0s 95us/step - loss: 0.0627 - accuracy: 0.9880 - f1: 0.9882 - val_loss: 0.1350 - val_accuracy: 0.9640 - val_f1: 0.9618
Epoch 4/100
4004/4004 [==============================] - 0s 95us/step - loss: 0.0355 - accuracy: 0.9948 - f1: 0.9944 - val_loss: 0.1372 - val_accuracy: 0.9540 - val_f1: 0.9558
Epoch 5/100
4004/4004 [===========

## 5k

In [0]:
number_of_attemp_to_fit = 1

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_5k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./Question_augmentation/TREC6_5k_bert_0.7__*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 39961 out of 108223 unique words
Average sentence len:  9.717636666666667
Max sentence len:  25601
5.955% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.379% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 270s 900us/step - loss: 0.4835 - accuracy: 0.8234 - f1: 0.8103 - val_loss: 0.4590 - val_accuracy: 0.8000 - val_f1: 0.8025
Best F1:0.802, best accuracy:0.800


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.192369157011585
Max sentence len:  37
6.381% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.379% of tokens were replaced by unk token index
Train on 5006 samples, validate on 500 samples
Epoch 1/100
5006/5006 [==============================] - 5s 935us/step - loss: 0.3860 - accuracy: 0.8646 - f1: 0.8641 - val_loss: 0.3112 - val_accuracy: 0.8920 - val_f1: 0.8912
Epoch 2/100
5006/5006 [==============================] - 5s 914us/step - loss: 0.3166 - accuracy: 0.8915 - f1: 0.8916 - val_loss: 0.2895 - val_accuracy: 0.8960 - val_f1: 0.8986
Epoch 3/100
5006/5006 [==============================] - 5s 926us/step - loss: 0.2707 - accuracy: 0.9099 - f1: 0.9104 - val_loss: 0.2605 - val_accuracy: 0.9100 - val_f1: 0.9124
Epoch 4/100
5006/5006 [==============================] - 5s 915us/step - loss: 0.2250 - accuracy: 0.9257 - f1: 0.9247 - val_loss: 0.2673 - val_accuracy: 0.9240 - val_f1: 0.9256
Epoch 5/100
5006/5006 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 39961 out of 108223 unique words
Average sentence len:  9.717636666666667
Max sentence len:  25601
5.955% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.379% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 27s 91us/step - loss: 0.3584 - accuracy: 0.8682 - f1: 0.8568 - val_loss: 0.2725 - val_accuracy: 0.9060 - val_f1: 0.9072
Best F1:0.907, best accuracy:0.906


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.192369157011585
Max sentence len:  37
6.381% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.379% of tokens were replaced by unk token index
Train on 5006 samples, validate on 500 samples
Epoch 1/100
5006/5006 [==============================] - 1s 115us/step - loss: 0.2263 - accuracy: 0.9279 - f1: 0.9271 - val_loss: 0.1519 - val_accuracy: 0.9660 - val_f1: 0.9648
Epoch 2/100
5006/5006 [==============================] - 0s 94us/step - loss: 0.1192 - accuracy: 0.9692 - f1: 0.9693 - val_loss: 0.1325 - val_accuracy: 0.9680 - val_f1: 0.9690
Epoch 3/100
5006/5006 [==============================] - 0s 94us/step - loss: 0.0604 - accuracy: 0.9870 - f1: 0.9866 - val_loss: 0.1430 - val_accuracy: 0.9620 - val_f1: 0.9648
Epoch 4/100
5006/5006 [==============================] - 0s 93us/step - loss: 0.0343 - accuracy: 0.9936 - f1: 0.9935 - val_loss: 0.1207 - val_accuracy: 0.9740 - val_f1: 0.9729
Epoch 5/100
5006/5006 [==========

# TREC-50 aug bert

## 3k

In [0]:
number_of_attemp_to_fit = 1

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_3k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./Question_augmentation/TREC50_3k_bert_0.7__*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 37930 out of 106794 unique words
Average sentence len:  9.77545
Max sentence len:  25601
5.303% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.406% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 271s 904us/step - loss: 1.0645 - accuracy: 0.7185 - f1: 0.7085 - val_loss: 1.6822 - val_accuracy: 0.7080 - val_f1: 0.6923
Best F1:0.692, best accuracy:0.708


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.240343347639485
Max sentence len:  32
6.254% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.406% of tokens were replaced by unk token index
Train on 3029 samples, validate on 500 samples
Epoch 1/100
3029/3029 [==============================] - 3s 954us/step - loss: 1.4528 - accuracy: 0.6659 - f1: 0.6415 - val_loss: 1.4816 - val_accuracy: 0.7080 - val_f1: 0.6625
Epoch 2/100
3029/3029 [==============================] - 3s 957us/step - loss: 1.2471 - accuracy: 0.7022 - f1: 0.6698 - val_loss: 1.2989 - val_accuracy: 0.7360 - val_f1: 0.6913
Epoch 3/100
3029/3029 [==============================] - 3s 938us/step - loss: 1.0672 - accuracy: 0.7389 - f1: 0.7162 - val_loss: 1.1568 - val_accuracy: 0.7600 - val_f1: 0.7486
Epoch 4/100
3029/3029 [==============================] - 3s 942us/step - loss: 0.9343 - accuracy: 0.7597 - f1: 0.7486 - val_loss: 1.0032 - val_accuracy: 0.7960 - val_f1: 0.7928
Epoch 5/100
3029/3029 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 37930 out of 106794 unique words
Average sentence len:  9.77545
Max sentence len:  25601
5.303% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.406% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 28s 92us/step - loss: 0.6300 - accuracy: 0.8255 - f1: 0.8171 - val_loss: 1.1564 - val_accuracy: 0.8340 - val_f1: 0.8436
Best F1:0.844, best accuracy:0.834


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.240343347639485
Max sentence len:  32
6.254% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.406% of tokens were replaced by unk token index
Train on 3029 samples, validate on 500 samples
Epoch 1/100
3029/3029 [==============================] - 1s 169us/step - loss: 0.7982 - accuracy: 0.8455 - f1: 0.8495 - val_loss: 0.8906 - val_accuracy: 0.8400 - val_f1: 0.8520
Epoch 2/100
3029/3029 [==============================] - 0s 100us/step - loss: 0.5041 - accuracy: 0.8973 - f1: 0.8984 - val_loss: 0.6642 - val_accuracy: 0.8720 - val_f1: 0.8803
Epoch 3/100
3029/3029 [==============================] - 0s 97us/step - loss: 0.3190 - accuracy: 0.9376 - f1: 0.9345 - val_loss: 0.5530 - val_accuracy: 0.8700 - val_f1: 0.8787
Epoch 4/100
3029/3029 [==============================] - 0s 96us/step - loss: 0.2051 - accuracy: 0.9614 - f1: 0.9612 - val_loss: 0.5003 - val_accuracy: 0.8860 - val_f1: 0.8963
Epoch 5/100
3029/3029 [=========

## 4k

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_4k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./Question_augmentation/TREC50_4k_bert_0.7__*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 38602 out of 107541 unique words
Average sentence len:  9.76348
Max sentence len:  25601
5.503% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.433% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 273s 908us/step - loss: 1.1151 - accuracy: 0.7030 - f1: 0.6899 - val_loss: 1.4083 - val_accuracy: 0.7340 - val_f1: 0.7211
Best F1:0.721, best accuracy:0.734


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.186639979984989
Max sentence len:  37
6.469% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.433% of tokens were replaced by unk token index
Train on 3997 samples, validate on 500 samples
Epoch 1/100
3997/3997 [==============================] - 4s 952us/step - loss: 1.1669 - accuracy: 0.7223 - f1: 0.7159 - val_loss: 1.1433 - val_accuracy: 0.7700 - val_f1: 0.7290
Epoch 2/100
3997/3997 [==============================] - 4s 919us/step - loss: 0.9604 - accuracy: 0.7598 - f1: 0.7533 - val_loss: 0.9513 - val_accuracy: 0.8040 - val_f1: 0.7978
Epoch 3/100
3997/3997 [==============================] - 4s 929us/step - loss: 0.8024 - accuracy: 0.7991 - f1: 0.7895 - val_loss: 0.8392 - val_accuracy: 0.8200 - val_f1: 0.8256
Epoch 4/100
3997/3997 [==============================] - 4s 964us/step - loss: 0.6869 - accuracy: 0.8136 - f1: 0.8181 - val_loss: 0.8343 - val_accuracy: 0.8200 - val_f1: 0.8104
Epoch 5/100
3997/3997 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 38602 out of 107541 unique words
Average sentence len:  9.76348
Max sentence len:  25601
5.503% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.433% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 27s 92us/step - loss: 0.6621 - accuracy: 0.8161 - f1: 0.8075 - val_loss: 0.9511 - val_accuracy: 0.8460 - val_f1: 0.8448
Best F1:0.845, best accuracy:0.846


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.186639979984989
Max sentence len:  37
6.469% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.433% of tokens were replaced by unk token index
Train on 3997 samples, validate on 500 samples
Epoch 1/100
3997/3997 [==============================] - 1s 126us/step - loss: 0.6211 - accuracy: 0.8624 - f1: 0.8644 - val_loss: 0.7304 - val_accuracy: 0.8780 - val_f1: 0.8779
Epoch 2/100
3997/3997 [==============================] - 0s 97us/step - loss: 0.3626 - accuracy: 0.9199 - f1: 0.9208 - val_loss: 0.5915 - val_accuracy: 0.8780 - val_f1: 0.8866
Epoch 3/100
3997/3997 [==============================] - 0s 95us/step - loss: 0.2153 - accuracy: 0.9575 - f1: 0.9558 - val_loss: 0.5188 - val_accuracy: 0.8880 - val_f1: 0.9000
Epoch 4/100
3997/3997 [==============================] - 0s 97us/step - loss: 0.1296 - accuracy: 0.9750 - f1: 0.9745 - val_loss: 0.5129 - val_accuracy: 0.8880 - val_f1: 0.9029
Epoch 5/100
3997/3997 [==========

## 5k

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_5k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./Question_augmentation/TREC50_5k_bert_0.7__*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 38687 out of 107462 unique words
Average sentence len:  9.754986666666667
Max sentence len:  25601
5.533% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.406% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 276s 921us/step - loss: 1.1597 - accuracy: 0.6919 - f1: 0.6723 - val_loss: 1.3174 - val_accuracy: 0.7180 - val_f1: 0.7004
Best F1:0.700, best accuracy:0.718


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.208824116590138
Max sentence len:  37
6.358% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.406% of tokens were replaced by unk token index
Train on 5009 samples, validate on 500 samples
Epoch 1/100
5009/5009 [==============================] - 5s 966us/step - loss: 1.0527 - accuracy: 0.7255 - f1: 0.7245 - val_loss: 1.0749 - val_accuracy: 0.7620 - val_f1: 0.7557
Epoch 2/100
5009/5009 [==============================] - 5s 950us/step - loss: 0.8960 - accuracy: 0.7648 - f1: 0.7655 - val_loss: 0.9760 - val_accuracy: 0.7820 - val_f1: 0.7852
Epoch 3/100
5009/5009 [==============================] - 5s 972us/step - loss: 0.7605 - accuracy: 0.8012 - f1: 0.7956 - val_loss: 0.9250 - val_accuracy: 0.7900 - val_f1: 0.7978
Epoch 4/100
5009/5009 [==============================] - 5s 951us/step - loss: 0.6564 - accuracy: 0.8205 - f1: 0.8202 - val_loss: 0.8926 - val_accuracy: 0.7800 - val_f1: 0.8148
Epoch 5/100
5009/5009 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs[:300000], test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 38687 out of 107462 unique words
Average sentence len:  9.754986666666667
Max sentence len:  25601
5.533% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.406% of tokens were replaced by unk token index
Train on 300000 samples, validate on 500 samples
Epoch 1/1
300000/300000 [==============================] - 28s 92us/step - loss: 0.6881 - accuracy: 0.8099 - f1: 0.7985 - val_loss: 0.8763 - val_accuracy: 0.8520 - val_f1: 0.8574
Best F1:0.857, best accuracy:0.852


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.208824116590138
Max sentence len:  37
6.358% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
3.406% of tokens were replaced by unk token index
Train on 5009 samples, validate on 500 samples
Epoch 1/100
5009/5009 [==============================] - 0s 94us/step - loss: 0.5207 - accuracy: 0.8754 - f1: 0.8730 - val_loss: 0.6837 - val_accuracy: 0.8780 - val_f1: 0.8725
Epoch 2/100
5009/5009 [==============================] - 0s 95us/step - loss: 0.2985 - accuracy: 0.9333 - f1: 0.9313 - val_loss: 0.5626 - val_accuracy: 0.8760 - val_f1: 0.8875
Epoch 3/100
5009/5009 [==============================] - 0s 96us/step - loss: 0.1668 - accuracy: 0.9675 - f1: 0.9674 - val_loss: 0.4930 - val_accuracy: 0.8880 - val_f1: 0.8914
Epoch 4/100
5009/5009 [==============================] - 0s 96us/step - loss: 0.0997 - accuracy: 0.9822 - f1: 0.9827 - val_loss: 0.4836 - val_accuracy: 0.8960 - val_f1: 0.8969
Epoch 5/100
5009/5009 [===========

# TREC-6 aug bert faiss

In [0]:
!ls

## 3k

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_3k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./USE_filtring_3k_*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 10527 out of 36905 unique words
Average sentence len:  8.895179521172649
Max sentence len:  33
6.460% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.360% of tokens were replaced by unk token index
Train on 319823 samples, validate on 500 samples
Epoch 1/1
319823/319823 [==============================] - 152s 475us/step - loss: 0.3288 - accuracy: 0.8838 - f1: 0.8752 - val_loss: 0.3163 - val_accuracy: 0.8980 - val_f1: 0.9008
Best F1:0.901, best accuracy:0.898


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.175099866844208
Max sentence len:  37
9.236% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.360% of tokens were replaced by unk token index
Train on 3004 samples, validate on 500 samples
Epoch 1/100
3004/3004 [==============================] - 2s 506us/step - loss: 0.3684 - accuracy: 0.8765 - f1: 0.8762 - val_loss: 0.2654 - val_accuracy: 0.9180 - val_f1: 0.9181
Epoch 2/100
3004/3004 [==============================] - 1s 476us/step - loss: 0.3013 - accuracy: 0.8988 - f1: 0.8977 - val_loss: 0.2171 - val_accuracy: 0.9320 - val_f1: 0.9359
Epoch 3/100
3004/3004 [==============================] - 1s 485us/step - loss: 0.2544 - accuracy: 0.9138 - f1: 0.9148 - val_loss: 0.2473 - val_accuracy: 0.9260 - val_f1: 0.9242
Epoch 4/100
3004/3004 [==============================] - 1s 473us/step - loss: 0.2042 - accuracy: 0.9334 - f1: 0.9327 - val_loss: 0.2185 - val_accuracy: 0.9360 - val_f1: 0.9356
Epoch 5/100
3004/3004 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 10527 out of 36905 unique words
Average sentence len:  8.895179521172649
Max sentence len:  33
6.460% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.360% of tokens were replaced by unk token index
Train on 319823 samples, validate on 500 samples
Epoch 1/1
319823/319823 [==============================] - 15s 46us/step - loss: 0.1495 - accuracy: 0.9487 - f1: 0.9386 - val_loss: 0.2189 - val_accuracy: 0.9400 - val_f1: 0.9378
Best F1:0.938, best accuracy:0.940


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.175099866844208
Max sentence len:  37
9.236% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.360% of tokens were replaced by unk token index
Train on 3004 samples, validate on 500 samples
Epoch 1/100
3004/3004 [==============================] - 0s 43us/step - loss: 0.3306 - accuracy: 0.9068 - f1: 0.9072 - val_loss: 0.2107 - val_accuracy: 0.9480 - val_f1: 0.9509
Epoch 2/100
3004/3004 [==============================] - 0s 34us/step - loss: 0.1030 - accuracy: 0.9667 - f1: 0.9672 - val_loss: 0.1785 - val_accuracy: 0.9540 - val_f1: 0.9518
Epoch 3/100
3004/3004 [==============================] - 0s 34us/step - loss: 0.0434 - accuracy: 0.9883 - f1: 0.9882 - val_loss: 0.2369 - val_accuracy: 0.9400 - val_f1: 0.9399
Epoch 4/100
3004/3004 [==============================] - 0s 33us/step - loss: 0.0238 - accuracy: 0.9940 - f1: 0.9939 - val_loss: 0.1945 - val_accuracy: 0.9520 - val_f1: 0.9498
Epoch 5/100
3004/3004 [===========

In [0]:
!ls

## 4k

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_4k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./USE_filtring_trec6_4k*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 12101 out of 41371 unique words
Average sentence len:  8.899900982627369
Max sentence len:  36
6.367% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.828% of tokens were replaced by unk token index
Train on 437297 samples, validate on 500 samples
Epoch 1/1
437297/437297 [==============================] - 209s 479us/step - loss: 0.2806 - accuracy: 0.9017 - f1: 0.8950 - val_loss: 0.2519 - val_accuracy: 0.9140 - val_f1: 0.9127
Best F1:0.913, best accuracy:0.914


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.211788211788212
Max sentence len:  37
8.787% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.828% of tokens were replaced by unk token index
Train on 4004 samples, validate on 500 samples
Epoch 1/100
4004/4004 [==============================] - 2s 516us/step - loss: 0.3375 - accuracy: 0.8854 - f1: 0.8874 - val_loss: 0.1965 - val_accuracy: 0.9280 - val_f1: 0.9333
Epoch 2/100
4004/4004 [==============================] - 2s 480us/step - loss: 0.2446 - accuracy: 0.9213 - f1: 0.9220 - val_loss: 0.1910 - val_accuracy: 0.9440 - val_f1: 0.9458
Epoch 3/100
4004/4004 [==============================] - 2s 498us/step - loss: 0.1900 - accuracy: 0.9391 - f1: 0.9395 - val_loss: 0.1943 - val_accuracy: 0.9420 - val_f1: 0.9448
Epoch 4/100
4004/4004 [==============================] - 2s 476us/step - loss: 0.1405 - accuracy: 0.9603 - f1: 0.9605 - val_loss: 0.2195 - val_accuracy: 0.9400 - val_f1: 0.9399
Epoch 5/100
4004/4004 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 12101 out of 41371 unique words
Average sentence len:  8.899900982627369
Max sentence len:  36
6.367% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.828% of tokens were replaced by unk token index
Train on 437297 samples, validate on 500 samples
Epoch 1/1
437297/437297 [==============================] - 14s 32us/step - loss: 0.1162 - accuracy: 0.9605 - f1: 0.9536 - val_loss: 0.2095 - val_accuracy: 0.9600 - val_f1: 0.9588
Best F1:0.959, best accuracy:0.960


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.211788211788212
Max sentence len:  37
8.787% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.828% of tokens were replaced by unk token index
Train on 4004 samples, validate on 500 samples
Epoch 1/100
4004/4004 [==============================] - 0s 40us/step - loss: 0.3088 - accuracy: 0.9128 - f1: 0.9143 - val_loss: 0.2355 - val_accuracy: 0.9500 - val_f1: 0.9499
Epoch 2/100
4004/4004 [==============================] - 0s 32us/step - loss: 0.0694 - accuracy: 0.9793 - f1: 0.9800 - val_loss: 0.1451 - val_accuracy: 0.9720 - val_f1: 0.9719
Epoch 3/100
4004/4004 [==============================] - 0s 34us/step - loss: 0.0271 - accuracy: 0.9938 - f1: 0.9939 - val_loss: 0.1831 - val_accuracy: 0.9700 - val_f1: 0.9700
Epoch 4/100
4004/4004 [==============================] - 0s 32us/step - loss: 0.0145 - accuracy: 0.9963 - f1: 0.9963 - val_loss: 0.1415 - val_accuracy: 0.9740 - val_f1: 0.9750
Epoch 5/100
4004/4004 [===========

## 5k

In [0]:
train_data, test_data, l2i = read_trec6('trec6_splited/trec6_splited_5k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./USE_filtring_trec6_5k*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 13259 out of 44601 unique words
Average sentence len:  8.897830424456007
Max sentence len:  33
6.432% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.695% of tokens were replaced by unk token index
Train on 531763 samples, validate on 500 samples
Epoch 1/1
531763/531763 [==============================] - 253s 476us/step - loss: 0.2615 - accuracy: 0.9092 - f1: 0.9040 - val_loss: 0.2325 - val_accuracy: 0.9320 - val_f1: 0.9263
Best F1:0.926, best accuracy:0.932


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.192369157011585
Max sentence len:  37
8.624% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.695% of tokens were replaced by unk token index
Train on 5006 samples, validate on 500 samples
Epoch 1/100
5006/5006 [==============================] - 3s 504us/step - loss: 0.3235 - accuracy: 0.8945 - f1: 0.8950 - val_loss: 0.2041 - val_accuracy: 0.9320 - val_f1: 0.9320
Epoch 2/100
5006/5006 [==============================] - 2s 481us/step - loss: 0.2403 - accuracy: 0.9189 - f1: 0.9205 - val_loss: 0.1849 - val_accuracy: 0.9420 - val_f1: 0.9416
Epoch 3/100
5006/5006 [==============================] - 2s 485us/step - loss: 0.1806 - accuracy: 0.9429 - f1: 0.9429 - val_loss: 0.1726 - val_accuracy: 0.9460 - val_f1: 0.9438
Epoch 4/100
5006/5006 [==============================] - 2s 483us/step - loss: 0.1397 - accuracy: 0.9573 - f1: 0.9580 - val_loss: 0.1759 - val_accuracy: 0.9500 - val_f1: 0.9510
Epoch 5/100
5006/5006 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 13259 out of 44601 unique words
Average sentence len:  8.897830424456007
Max sentence len:  33
6.432% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.695% of tokens were replaced by unk token index
Train on 531763 samples, validate on 500 samples
Epoch 1/1
531763/531763 [==============================] - 17s 32us/step - loss: 0.1069 - accuracy: 0.9644 - f1: 0.9588 - val_loss: 0.1959 - val_accuracy: 0.9460 - val_f1: 0.9449
Best F1:0.945, best accuracy:0.946


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.192369157011585
Max sentence len:  37
8.624% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.695% of tokens were replaced by unk token index
Train on 5006 samples, validate on 500 samples
Epoch 1/100
5006/5006 [==============================] - 0s 33us/step - loss: 0.2977 - accuracy: 0.9139 - f1: 0.9140 - val_loss: 0.2088 - val_accuracy: 0.9520 - val_f1: 0.9509
Epoch 2/100
5006/5006 [==============================] - 0s 33us/step - loss: 0.0650 - accuracy: 0.9816 - f1: 0.9820 - val_loss: 0.1518 - val_accuracy: 0.9640 - val_f1: 0.9640
Epoch 3/100
5006/5006 [==============================] - 0s 33us/step - loss: 0.0237 - accuracy: 0.9946 - f1: 0.9946 - val_loss: 0.1434 - val_accuracy: 0.9540 - val_f1: 0.9550
Epoch 4/100
5006/5006 [==============================] - 0s 33us/step - loss: 0.0139 - accuracy: 0.9966 - f1: 0.9967 - val_loss: 0.1396 - val_accuracy: 0.9680 - val_f1: 0.9670
Epoch 5/100
5006/5006 [===========

# TREC-50 aug bert faiss

## 3k

In [0]:
!ls

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_3k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./USE_filtring_trec50_3k_*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 9406 out of 32921 unique words
Average sentence len:  8.937510601928572
Max sentence len:  31
6.157% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.919% of tokens were replaced by unk token index
Train on 383185 samples, validate on 500 samples
Epoch 1/1
383185/383185 [==============================] - 184s 481us/step - loss: 0.7831 - accuracy: 0.7907 - f1: 0.7782 - val_loss: 1.4505 - val_accuracy: 0.8060 - val_f1: 0.8090
Best F1:0.809, best accuracy:0.806


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.240343347639485
Max sentence len:  32
9.572% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.919% of tokens were replaced by unk token index
Train on 3029 samples, validate on 500 samples
Epoch 1/100
3029/3029 [==============================] - 1s 484us/step - loss: 1.0970 - accuracy: 0.7838 - f1: 0.7971 - val_loss: 0.9491 - val_accuracy: 0.8260 - val_f1: 0.8273
Epoch 2/100
3029/3029 [==============================] - 1s 482us/step - loss: 0.8126 - accuracy: 0.8181 - f1: 0.8162 - val_loss: 0.8580 - val_accuracy: 0.8280 - val_f1: 0.8170
Epoch 3/100
3029/3029 [==============================] - 2s 501us/step - loss: 0.6690 - accuracy: 0.8366 - f1: 0.8363 - val_loss: 0.7538 - val_accuracy: 0.8300 - val_f1: 0.8457
Epoch 4/100
3029/3029 [==============================] - 1s 477us/step - loss: 0.5478 - accuracy: 0.8640 - f1: 0.8670 - val_loss: 0.6920 - val_accuracy: 0.8380 - val_f1: 0.8494
Epoch 5/100
3029/3029 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 9406 out of 32921 unique words
Average sentence len:  8.937510601928572
Max sentence len:  31
6.157% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.919% of tokens were replaced by unk token index
Train on 383185 samples, validate on 500 samples
Epoch 1/1
383185/383185 [==============================] - 12s 31us/step - loss: 0.2917 - accuracy: 0.9254 - f1: 0.9157 - val_loss: 1.6292 - val_accuracy: 0.8420 - val_f1: 0.8528
Best F1:0.853, best accuracy:0.842


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.240343347639485
Max sentence len:  32
9.572% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.919% of tokens were replaced by unk token index
Train on 3029 samples, validate on 500 samples
Epoch 1/100
3029/3029 [==============================] - 0s 34us/step - loss: 0.9999 - accuracy: 0.8392 - f1: 0.8483 - val_loss: 1.1355 - val_accuracy: 0.8360 - val_f1: 0.8504
Epoch 2/100
3029/3029 [==============================] - 0s 33us/step - loss: 0.3823 - accuracy: 0.9204 - f1: 0.9264 - val_loss: 0.7142 - val_accuracy: 0.8440 - val_f1: 0.8607
Epoch 3/100
3029/3029 [==============================] - 0s 33us/step - loss: 0.1872 - accuracy: 0.9607 - f1: 0.9591 - val_loss: 0.6082 - val_accuracy: 0.8620 - val_f1: 0.8792
Epoch 4/100
3029/3029 [==============================] - 0s 36us/step - loss: 0.1023 - accuracy: 0.9766 - f1: 0.9752 - val_loss: 0.6452 - val_accuracy: 0.8560 - val_f1: 0.8693
Epoch 5/100
3029/3029 [===========

## 4k

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_4k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./USE_filtring_trec50_4k_*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 11193 out of 38025 unique words
Average sentence len:  8.881059186959808
Max sentence len:  36
6.930% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.253% of tokens were replaced by unk token index
Train on 365148 samples, validate on 500 samples
Epoch 1/1
365148/365148 [==============================] - 176s 481us/step - loss: 0.8536 - accuracy: 0.7720 - f1: 0.7599 - val_loss: 1.0332 - val_accuracy: 0.8260 - val_f1: 0.8374
Best F1:0.837, best accuracy:0.826


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.186639979984989
Max sentence len:  37
9.222% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.253% of tokens were replaced by unk token index
Train on 3997 samples, validate on 500 samples
Epoch 1/100
3997/3997 [==============================] - 2s 508us/step - loss: 0.8165 - accuracy: 0.8159 - f1: 0.8239 - val_loss: 0.8052 - val_accuracy: 0.8420 - val_f1: 0.8351
Epoch 2/100
3997/3997 [==============================] - 2s 510us/step - loss: 0.6504 - accuracy: 0.8389 - f1: 0.8368 - val_loss: 0.6619 - val_accuracy: 0.8540 - val_f1: 0.8611
Epoch 3/100
3997/3997 [==============================] - 2s 498us/step - loss: 0.5316 - accuracy: 0.8614 - f1: 0.8619 - val_loss: 0.6178 - val_accuracy: 0.8580 - val_f1: 0.8678
Epoch 4/100
3997/3997 [==============================] - 2s 497us/step - loss: 0.4422 - accuracy: 0.8832 - f1: 0.8827 - val_loss: 0.6143 - val_accuracy: 0.8460 - val_f1: 0.8607
Epoch 5/100
3997/3997 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 11193 out of 38025 unique words
Average sentence len:  8.881059186959808
Max sentence len:  36
6.930% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.253% of tokens were replaced by unk token index
Train on 365148 samples, validate on 500 samples
Epoch 1/1
365148/365148 [==============================] - 12s 32us/step - loss: 0.3309 - accuracy: 0.9146 - f1: 0.9054 - val_loss: 1.1233 - val_accuracy: 0.8460 - val_f1: 0.8536
Best F1:0.854, best accuracy:0.846


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.186639979984989
Max sentence len:  37
9.222% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
6.253% of tokens were replaced by unk token index
Train on 3997 samples, validate on 500 samples
Epoch 1/100
3997/3997 [==============================] - 0s 40us/step - loss: 0.7129 - accuracy: 0.8639 - f1: 0.8697 - val_loss: 0.8089 - val_accuracy: 0.8800 - val_f1: 0.8805
Epoch 2/100
3997/3997 [==============================] - 0s 32us/step - loss: 0.2318 - accuracy: 0.9455 - f1: 0.9502 - val_loss: 0.6450 - val_accuracy: 0.8580 - val_f1: 0.8669
Epoch 3/100
3997/3997 [==============================] - 0s 32us/step - loss: 0.1006 - accuracy: 0.9780 - f1: 0.9795 - val_loss: 0.5982 - val_accuracy: 0.8600 - val_f1: 0.8669
Epoch 4/100
3997/3997 [==============================] - 0s 34us/step - loss: 0.0542 - accuracy: 0.9887 - f1: 0.9876 - val_loss: 0.5690 - val_accuracy: 0.8780 - val_f1: 0.8882
Epoch 5/100
3997/3997 [===========

## 5k

In [0]:
train_data, test_data, l2i = read_trec50('trec50_splited/trec50_5k.csv')

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
from glob import glob 
import pandas as pd
files = glob('./USE_filtring_trec50_5k_*')
all_augs = pd.concat([pd.read_csv(i, index_col=0) for i in files])

In [0]:
f1_score, acc_score, aug_lstm, w2i = re_fit_lstm(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 12493 out of 41393 unique words
Average sentence len:  8.866279771675474
Max sentence len:  36
6.867% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.854% of tokens were replaced by unk token index
Train on 472135 samples, validate on 500 samples
Epoch 1/1
472135/472135 [==============================] - 227s 480us/step - loss: 0.7227 - accuracy: 0.8054 - f1: 0.7975 - val_loss: 0.9395 - val_accuracy: 0.8480 - val_f1: 0.8566
Best F1:0.857, best accuracy:0.848


In [0]:
f1_score, acc_score, res_lstm, _ = re_fit_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_lstm, word2ind_aug=w2i)

Average sentence len:  10.208824116590138
Max sentence len:  37
8.976% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.854% of tokens were replaced by unk token index
Train on 5009 samples, validate on 500 samples
Epoch 1/100
5009/5009 [==============================] - 2s 494us/step - loss: 0.6814 - accuracy: 0.8385 - f1: 0.8456 - val_loss: 0.7570 - val_accuracy: 0.8460 - val_f1: 0.8568
Epoch 2/100
5009/5009 [==============================] - 2s 483us/step - loss: 0.5288 - accuracy: 0.8607 - f1: 0.8644 - val_loss: 0.5969 - val_accuracy: 0.8640 - val_f1: 0.8641
Epoch 3/100
5009/5009 [==============================] - 2s 489us/step - loss: 0.4278 - accuracy: 0.8808 - f1: 0.8856 - val_loss: 0.5444 - val_accuracy: 0.8640 - val_f1: 0.8715
Epoch 4/100
5009/5009 [==============================] - 2s 488us/step - loss: 0.3508 - accuracy: 0.9130 - f1: 0.9128 - val_loss: 0.5786 - val_accuracy: 0.8620 - val_f1: 0.8697
Epoch 5/100
5009/5009 [=======

In [0]:
f1_score, acc_score, aug_cnn, w2i = re_fit_cnn(all_augs, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, epochs=1)

Not found vectors 12493 out of 41393 unique words
Average sentence len:  8.866279771675474
Max sentence len:  36
6.867% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.854% of tokens were replaced by unk token index
Train on 472135 samples, validate on 500 samples
Epoch 1/1
472135/472135 [==============================] - 15s 32us/step - loss: 0.2774 - accuracy: 0.9286 - f1: 0.9212 - val_loss: 1.1222 - val_accuracy: 0.8640 - val_f1: 0.8650
Best F1:0.865, best accuracy:0.864


In [0]:
f1_score, acc_score, res_cnn, _ = re_fit_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES, model=aug_cnn, word2ind_aug=w2i)

Average sentence len:  10.208824116590138
Max sentence len:  37
8.976% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
5.854% of tokens were replaced by unk token index
Train on 5009 samples, validate on 500 samples
Epoch 1/100
5009/5009 [==============================] - 0s 39us/step - loss: 0.6139 - accuracy: 0.8646 - f1: 0.8712 - val_loss: 0.7894 - val_accuracy: 0.8620 - val_f1: 0.8755
Epoch 2/100
5009/5009 [==============================] - 0s 33us/step - loss: 0.1853 - accuracy: 0.9565 - f1: 0.9585 - val_loss: 0.5796 - val_accuracy: 0.8780 - val_f1: 0.8850
Epoch 3/100
5009/5009 [==============================] - 0s 33us/step - loss: 0.0754 - accuracy: 0.9818 - f1: 0.9829 - val_loss: 0.5477 - val_accuracy: 0.8980 - val_f1: 0.9000
Epoch 4/100
5009/5009 [==============================] - 0s 33us/step - loss: 0.0381 - accuracy: 0.9928 - f1: 0.9908 - val_loss: 0.5265 - val_accuracy: 0.9020 - val_f1: 0.9085
Epoch 5/100
5009/5009 [===========